<a href="https://colab.research.google.com/github/huckleberryCalfinn/manim/blob/master/RC_Starter_Analysis_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
"""
Starter Batch Analysis System - Restructured for Batch-Centric Analysis
Author: Process Improvement Team
Description: Interactive analysis focused on starter batches and their performance across intermediates
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import widgets for interactivity
import ipywidgets as widgets
from IPython.display import display, clear_output

# Set up plotting style
plt.style.use('default')
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.4
plt.rcParams['grid.color'] = 'gray'
sns.set_palette("husl")

class StarterBatchAnalyzer:
    def __init__(self, file_path):
        """Initialize the analyzer with data from Excel file"""
        self.df = pd.read_excel(file_path)
        self.prepare_data()
        self.setup_widgets()

    def prepare_data(self):
        """Clean and prepare data for batch-centric analysis"""
        # Convert date columns
        self.df['DOM'] = pd.to_datetime(self.df['DOM'])

        # Extract starter batch information from Starter Batch ID
        self.df['Starter_Made_Date'] = self.df['Starter Batch ID'].apply(self.extract_starter_date)
        self.df['Starter_Tank'] = self.df['Starter Batch ID'].apply(self.extract_starter_tank)
        self.df['Strain_Number'] = self.df['Starter Batch ID'].apply(self.extract_strain_number)

        # Convert time columns to minutes for easier analysis
        time_cols = ['TFI', 'Avg IIT', 'Stdev IIT', 'Min IIT', 'Max IIT']
        for col in time_cols:
            if col in self.df.columns:
                self.df[f'{col}_minutes'] = self.df[col].apply(self.time_to_minutes)

        # Remove rows with missing critical data
        self.df = self.df.dropna(subset=['DOM', 'Starter Strain', 'Starter Batch ID'])

        # Convert strain names to strings for consistent handling
        self.df['Starter Strain'] = self.df['Starter Strain'].astype(str)

        # Create intermediate identifier if columns exist
        if 'IM Item No' in self.df.columns and 'IM Item Desc' in self.df.columns:
            self.df['Intermediate'] = self.df['IM Item No'].astype(str) + ' - ' + self.df['IM Item Desc'].astype(str)
        else:
            self.df['Intermediate'] = 'Unknown Intermediate'

    def extract_starter_date(self, batch_id):
        """Extract starter made date from Batch ID
        Format: 1072601091504 = 107 + 260109 + 15 + 04
        Positions 3-8 (YYMMDD): 260109 = 2026-01-09"""
        try:
            if pd.isna(batch_id):
                return None
            batch_str = str(int(float(batch_id)))  # Handle potential float values
            if len(batch_str) >= 9:  # Need at least 9 digits to extract date
                year = int('20' + batch_str[3:5])  # Positions 3-4: YY -> 20YY
                month = int(batch_str[5:7])        # Positions 5-6: MM
                day = int(batch_str[7:9])          # Positions 7-8: DD
                # Validate date components
                if 1 <= month <= 12 and 1 <= day <= 31 and 2020 <= year <= 2030:
                    return pd.to_datetime(f'{year}-{month:02d}-{day:02d}')
            return None
        except (ValueError, TypeError, IndexError):
            return None

    def extract_starter_tank(self, batch_id):
        """Extract starter tank from Batch ID (positions 9-10)"""
        try:
            if pd.isna(batch_id):
                return None
            batch_str = str(int(float(batch_id)))  # Handle potential float values
            if len(batch_str) >= 11:
                return int(batch_str[9:11])  # Positions 9-10
            return None
        except (ValueError, TypeError, IndexError):
            return None

    def extract_strain_number(self, batch_id):
        """Extract strain number from Batch ID (positions 11-12)"""
        try:
            if pd.isna(batch_id):
                return None
            batch_str = str(int(float(batch_id)))  # Handle potential float values
            if len(batch_str) >= 13:
                return int(batch_str[11:13])  # Positions 11-12
            return None
        except (ValueError, TypeError, IndexError):
            return None

    def time_to_minutes(self, time_val):
        """Convert time format to minutes"""
        if pd.isna(time_val):
            return np.nan
        try:
            # Handle string format like "0:06:54"
            if isinstance(time_val, str):
                time_val = time_val.strip()
                parts = time_val.split(':')
                if len(parts) == 3:
                    hours = int(parts[0])
                    minutes = int(parts[1])
                    seconds = int(parts[2])
                    return hours * 60 + minutes + seconds / 60
                else:
                    return np.nan
            # Handle datetime objects
            elif isinstance(time_val, datetime):
                return time_val.hour * 60 + time_val.minute + time_val.second / 60
            # Handle pandas Timedelta objects
            elif hasattr(time_val, 'total_seconds'):
                return time_val.total_seconds() / 60
            else:
                # Try to convert to string and parse
                time_str = str(time_val)
                if ':' in time_str:
                    parts = time_str.split(':')
                    if len(parts) >= 2:
                        hours = int(parts[0]) if parts[0].isdigit() else 0
                        minutes = int(parts[1]) if parts[1].isdigit() else 0
                        seconds = int(parts[2]) if len(parts) > 2 and parts[2].isdigit() else 0
                        return hours * 60 + minutes + seconds / 60
                return np.nan
        except (ValueError, IndexError, AttributeError) as e:
            print(f"Warning: Could not parse time value '{time_val}': {e}")
            return np.nan

    def debug_batch_ids(self):
        """Debug function to check Batch ID parsing"""
        print("🔍 BATCH ID PARSING DEBUG")
        print("-" * 35)

        print("📊 Sample Batch IDs and parsing results:")
        sample_batch_ids = self.df['Starter Batch ID'].dropna().head(10)

        for i, batch_id in enumerate(sample_batch_ids, 1):
            try:
                batch_str = str(int(float(batch_id)))
                date_extracted = self.extract_starter_date(batch_id)
                tank_extracted = self.extract_starter_tank(batch_id)
                strain_extracted = self.extract_strain_number(batch_id)

                print(f"  {i:2d}. Batch ID: {batch_id}")
                print(f"      → String: '{batch_str}' (length: {len(batch_str)})")
                print(f"      → Date: {date_extracted}")
                print(f"      → Tank: {tank_extracted}")
                print(f"      → Strain: {strain_extracted}")
                print()
            except Exception as e:
                print(f"  {i:2d}. Batch ID: {batch_id} → ERROR: {e}")
                print()

        # Summary stats
        total_ids = len(self.df['Starter Batch ID'].dropna())
        valid_dates = len(self.df['Starter_Made_Date'].dropna())

        print(f"📈 SUMMARY:")
        print(f"  Total Batch IDs: {total_ids}")
        print(f"  Valid dates extracted: {valid_dates}")
        print(f"  Success rate: {valid_dates/total_ids*100:.1f}%" if total_ids > 0 else "  Success rate: 0%")
        print()

    def setup_widgets(self):
        """Setup interactive widgets for starter batch selection"""
        # Get available starter made dates - filter out None values and ensure datetime type
        starter_dates = self.df['Starter_Made_Date'].dropna()

        # Additional check to ensure we have datetime objects
        valid_datetime_dates = []
        for date in starter_dates:
            if pd.notna(date) and isinstance(date, (pd.Timestamp, datetime)):
                valid_datetime_dates.append(date)

        if len(valid_datetime_dates) == 0:
            print("⚠️ Warning: No valid starter dates found in Batch IDs")
            print("Check that Starter Batch ID column contains valid batch numbers")
            self._create_empty_widgets()
            return

        # Convert to pandas Series for proper handling
        valid_dates_series = pd.Series(valid_datetime_dates)
        available_dates = sorted(valid_dates_series.dt.date.unique())

        # Date selection widget
        self.date_widget = widgets.Dropdown(
            options=[(date.strftime('%Y-%m-%d (%A)'), date) for date in available_dates],
            value=available_dates[0] if available_dates else None,
            description='Starter Made:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )

        # Strain selection widget (initially empty)
        self.strain_widget = widgets.Dropdown(
            options=[],
            description='Select Strain:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )

        # Generate report button
        self.generate_button = widgets.Button(
            description='🧪 Generate Batch Report',
            button_style='primary',
            layout=widgets.Layout(width='250px', height='40px')
        )

        # Output area for the report
        self.output = widgets.Output()

        # Set up event handlers
        self.date_widget.observe(self.on_date_change, names='value')
        self.generate_button.on_click(self.on_generate_click)

        # Initialize strain options for the first date
        if available_dates:
            self.update_strain_options()

    def _create_empty_widgets(self):
        """Create empty widgets when no valid data is found"""
        self.date_widget = widgets.Dropdown(
            options=[],
            description='Starter Made:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        self.strain_widget = widgets.Dropdown(
            options=[],
            description='Select Strain:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        self.generate_button = widgets.Button(
            description='🧪 Generate Batch Report',
            button_style='primary',
            layout=widgets.Layout(width='250px', height='40px')
        )
        self.output = widgets.Output()

    def on_date_change(self, change):
        """Handle starter made date selection change"""
        self.update_strain_options()

    def update_strain_options(self):
        """Update strain options based on selected starter made date"""
        if not self.date_widget.value:
            return

        selected_date = pd.to_datetime(self.date_widget.value)

        # Get strains that had starter made on the selected date
        date_batches = self.df[self.df['Starter_Made_Date'] == selected_date]
        available_strains = sorted(date_batches['Starter Strain'].unique())

        # Update strain widget options
        self.strain_widget.options = [(strain, strain) for strain in available_strains]

        # Set default value if options exist
        if available_strains:
            self.strain_widget.value = available_strains[0]

        # Clear previous output and show available info
        with self.output:
            clear_output(wait=True)
            if available_strains:
                print(f"📅 Starter batches made on {selected_date.strftime('%Y-%m-%d')}:")
                batch_info = date_batches[['Starter Strain', 'Starter_Tank', 'Starter Batch ID']].drop_duplicates()
                for _, row in batch_info.iterrows():
                    print(f"   🧪 Strain {row['Starter Strain']} - Tank {row['Starter_Tank']} - Batch {row['Starter Batch ID']}")
                print(f"\n💭 {len(available_strains)} strain(s) available for analysis")
            else:
                print(f"⚠️  No starter batches found for {selected_date.strftime('%Y-%m-%d')}")

    def on_generate_click(self, button):
        """Handle generate report button click"""
        with self.output:
            clear_output(wait=True)

            if not self.strain_widget.options:
                print("❌ No strain selected. Please choose a date with available starter batches.")
                return

            # Get selected values
            starter_made_date = pd.to_datetime(self.date_widget.value)
            strain_name = self.strain_widget.value

            # Generate the report
            self.generate_batch_report(starter_made_date, strain_name)

    def display_interface(self):
        """Display the interactive interface"""
        print("🧪 STARTER BATCH PERFORMANCE ANALYZER")
        print("="*60)
        print("🎯 New Analysis Focus:")
        print("1. Select when the starter was MADE")
        print("2. Choose which strain was made that day")
        print("3. Analyze performance across ALL intermediates using that starter")
        print("="*60)

        # Create interface layout
        controls = widgets.VBox([
            widgets.HBox([self.date_widget, self.strain_widget]),
            widgets.HBox([self.generate_button]),
            self.output
        ])

        display(controls)

    def generate_batch_report(self, starter_made_date, strain_name):
        """Generate comprehensive report for starter batch performance"""
        print(f"🧪 STARTER BATCH PERFORMANCE REPORT")
        print(f"{'='*70}")
        print(f"Starter Made Date: {starter_made_date.strftime('%Y-%m-%d (%A)')}")
        print(f"Strain: {strain_name}")
        print(f"Report Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"{'='*70}\n")

        # Get the specific starter batch(es) made on this date
        batch_info = self.df[
            (self.df['Starter_Made_Date'] == starter_made_date) &
            (self.df['Starter Strain'] == strain_name)
        ]

        if batch_info.empty:
            print(f"❌ No starter batches found for strain '{strain_name}' made on {starter_made_date.strftime('%Y-%m-%d')}")
            return

        # Get unique batch IDs for this strain/date combination
        batch_ids = batch_info['Starter Batch ID'].unique()

        # Get all lots that used any of these starter batches
        all_usage_data = self.df[self.df['Starter Batch ID'].isin(batch_ids)]

        print(f"📊 BATCH OVERVIEW")
        print("-" * 20)
        for batch_id in batch_ids:
            batch_specific = all_usage_data[all_usage_data['Starter Batch ID'] == batch_id]
            tank = batch_specific['Starter_Tank'].iloc[0] if len(batch_specific) > 0 else 'Unknown'
            usage_count = len(batch_specific)
            date_range = f"{batch_specific['DOM'].min().strftime('%m-%d')} to {batch_specific['DOM'].max().strftime('%m-%d')}" if usage_count > 1 else batch_specific['DOM'].iloc[0].strftime('%m-%d')
            print(f"  🆔 Batch {batch_id} (Tank {tank}): Used in {usage_count} lots from {date_range}")

        # Show intermediate breakdown
        if 'Intermediate' in all_usage_data.columns:
            intermediate_summary = all_usage_data.groupby('Intermediate').size().sort_values(ascending=False)
            print(f"\n🏭 INTERMEDIATE USAGE")
            print("-" * 20)
            for intermediate, count in intermediate_summary.items():
                print(f"  📦 {intermediate}: {count} lots")

        print(f"\n📈 Total lots analyzed: {len(all_usage_data)}")
        print(f"🗓️  Production date range: {all_usage_data['DOM'].min().strftime('%Y-%m-%d')} to {all_usage_data['DOM'].max().strftime('%Y-%m-%d')}")
        print()

        # Generate growth metrics (from the batch creation)
        growth_data = batch_info.iloc[0]  # Take first batch record for growth metrics
        self.growth_metrics_summary(growth_data)

        # Get historical data for this strain for comparison
        strain_historical = self.df[self.df['Starter Strain'] == strain_name]
        self.historical_comparison(growth_data, strain_historical)

        # Add growth metrics trends over time for this strain
        self.growth_metrics_trends(strain_historical, strain_name, starter_made_date)

        # Analyze performance across intermediates
        self.performance_by_intermediate(all_usage_data, strain_name, starter_made_date)

        # Overall performance charts
        self.pH_performance_analysis(all_usage_data, strain_name, starter_made_date)
        self.pH_distribution_analysis(all_usage_data, strain_name, starter_made_date)
        self.moisture_performance_analysis(all_usage_data, strain_name, starter_made_date)

        # Add process metrics by vat for the usage data
        self.process_metrics_by_vat(all_usage_data, strain_name, starter_made_date)

    def growth_metrics_summary(self, batch_row):
        """Display growth metrics summary for the starter batch in table format"""
        print("📊 STARTER GROWTH METRICS")
        print("-" * 30)

        metrics = {
            'Time to First Injection': f"{batch_row.get('TFI_minutes', 'N/A'):.1f} minutes" if pd.notna(batch_row.get('TFI_minutes')) else "N/A",
            'Average Interinjection Time': f"{batch_row.get('Avg. IIT_minutes', 'N/A'):.1f} minutes" if pd.notna(batch_row.get('Avg. IIT_minutes')) else "N/A",
            'Stdev Interinjection Time': f"{batch_row.get('Stdev IIT_minutes', 'N/A'):.1f} minutes" if pd.notna(batch_row.get('Stdev IIT_minutes')) else "N/A",
            'Min Interinjection Time': f"{batch_row.get('Min IIT_minutes', 'N/A'):.1f} minutes" if pd.notna(batch_row.get('Min IIT_minutes')) else "N/A",
            'Max Interinjection Time': f"{batch_row.get('Max IIT_minutes', 'N/A'):.1f} minutes" if pd.notna(batch_row.get('Max IIT_minutes')) else "N/A",
            'pH Change After Last Injection': f"{batch_row.get('pH CALI', 'N/A'):.2f}" if pd.notna(batch_row.get('pH CALI')) else "N/A"
        }

        # Create a formatted table
        print(f"{'Metric':<35} {'Value':<15}")
        print("-" * 50)
        for metric, value in metrics.items():
            print(f"{metric:<35} {value:<15}")
        print()

    def historical_comparison(self, batch_row, historical_data):
        """Compare current batch to historical strain performance"""
        print("📈 HISTORICAL COMPARISON")
        print("-" * 25)

        metrics_to_compare = {
            'TFI_minutes': 'Time to First Injection',
            'Avg. IIT_minutes': 'Average Interinjection Time',
            'Stdev IIT_minutes': 'Stdev Interinjection Time',
            'Min IIT_minutes': 'Min Interinjection Time',
            'Max IIT_minutes': 'Max Interinjection Time',
            'pH CALI': 'pH Change After Last Injection'
        }

        for metric_col, metric_name in metrics_to_compare.items():
            current_value = batch_row.get(metric_col)

            # For pH metrics, filter out zeros from historical data
            if 'pH' in metric_col:
                historical_values = historical_data[metric_col].dropna()
                historical_values = historical_values[historical_values != 0]
            else:
                historical_values = historical_data[metric_col].dropna()

            historical_avg = historical_values.mean()
            historical_std = historical_values.std()

            if pd.notna(current_value) and pd.notna(historical_avg) and current_value != 0:
                difference = current_value - historical_avg
                z_score = difference / historical_std if pd.notna(historical_std) and historical_std > 0 else 0

                # Process-specific flagging logic
                if 'IIT' in metric_col:  # Interinjection timing - only flag if too long
                    status = "🟢" if z_score < 1 else "🟡" if z_score < 2 else "🔴"
                elif metric_col == 'TFI_minutes':  # Time to First Injection
                    if z_score <= -5:  # Much too short
                        status = "🟡"
                    elif z_score < 1:  # Normal or slightly short
                        status = "🟢"
                    elif z_score < 2:  # Too long
                        status = "🟡"
                    else:  # Much too long
                        status = "🔴"
                elif metric_col == 'pH CALI':  # pH Change After Last Injection
                    if current_value >= -0.10:  # pH change >= -0.10 is bad
                        status = "🔴"
                    elif current_value > historical_avg:  # Current is higher (less negative) than average
                        if z_score > 2.0:  # Much higher (worse) than average
                            status = "🔴"
                        elif z_score > 1.0:  # Somewhat higher (worse) than average
                            status = "🟡"
                        else:  # Slightly higher than average but still acceptable
                            status = "🟢"
                    else:  # Current is lower (more negative) than average - potentially good
                        if z_score < -5.0:  # Much more negative than usual - flag as unusual
                            status = "🟡"
                        else:  # More negative than average - this is good!
                            status = "🟢"
                else:  # Default two-sided test
                    status = "🟢" if abs(z_score) < 1 else "🟡" if abs(z_score) < 2 else "🔴"

    def historical_comparison(self, batch_row, historical_data):
        """Compare current batch to historical strain performance in table format"""
        print("📈 HISTORICAL COMPARISON")
        print("-" * 25)

        metrics_to_compare = {
            'TFI_minutes': 'Time to First Injection',
            'Avg. IIT_minutes': 'Average Interinjection Time',
            'Stdev IIT_minutes': 'Stdev Interinjection Time',
            'Min IIT_minutes': 'Min Interinjection Time',
            'Max IIT_minutes': 'Max Interinjection Time',
            'pH CALI': 'pH Change After Last Injection'
        }

        # Create table headers
        print(f"{'Metric':<30} {'Current':<10} {'Hist Avg':<10} {'Hist Std':<10} {'Diff':<10} {'Z-Score':<10} {'Status':<6}")
        print("-" * 86)

        for metric_col, metric_name in metrics_to_compare.items():
            current_value = batch_row.get(metric_col)

            # For pH metrics, filter out zeros from historical data
            if 'pH' in metric_col:
                historical_values = historical_data[metric_col].dropna()
                historical_values = historical_values[historical_values != 0]
            else:
                historical_values = historical_data[metric_col].dropna()

            historical_avg = historical_values.mean()
            historical_std = historical_values.std()

            if pd.notna(current_value) and pd.notna(historical_avg) and current_value != 0:
                difference = current_value - historical_avg
                z_score = difference / historical_std if pd.notna(historical_std) and historical_std > 0 else 0

                # Process-specific flagging logic
                if 'IIT' in metric_col:  # Interinjection timing - only flag if too long
                    status = "🟢" if z_score < 1 else "🟡" if z_score < 2 else "🔴"
                elif metric_col == 'TFI_minutes':  # Time to First Injection
                    if z_score <= -5:  # Much too short
                        status = "🟡"
                    elif z_score < 1:  # Normal or slightly short
                        status = "🟢"
                    elif z_score < 2:  # Too long
                        status = "🟡"
                    else:  # Much too long
                        status = "🔴"
                elif metric_col == 'pH CALI':  # pH Change After Last Injection
                    if current_value >= -0.10:  # pH change >= -0.10 is bad
                        status = "🔴"
                    elif current_value > historical_avg:  # Current is higher (less negative) than average
                        if z_score > 2.0:  # Much higher (worse) than average
                            status = "🔴"
                        elif z_score > 1.0:  # Somewhat higher (worse) than average
                            status = "🟡"
                        else:  # Slightly higher than average but still acceptable
                            status = "🟢"
                    else:  # Current is lower (more negative) than average - potentially good
                        if z_score < -5.0:  # Much more negative than usual - flag as unusual
                            status = "🟡"
                        else:  # More negative than average - this is good!
                            status = "🟢"
                else:  # Default two-sided test
                    status = "🟢" if abs(z_score) < 1 else "🟡" if abs(z_score) < 2 else "🔴"

                # Format the table row
                short_metric = metric_name[:29] if len(metric_name) > 29 else metric_name
                print(f"{short_metric:<30} {current_value:<10.2f} {historical_avg:<10.2f} {historical_std:<10.2f} {difference:<+10.2f} {z_score:<10.2f} {status:<6}")

                if 'pH' in metric_col:
                    print(f"{'  (Based on ' + str(len(historical_values)) + ' non-zero measurements)':<86}")
            else:
                short_metric = metric_name[:29] if len(metric_name) > 29 else metric_name
                print(f"{short_metric:<30} {'N/A':<10} {'N/A':<10} {'N/A':<10} {'N/A':<10} {'N/A':<10} {'❌':<6}")
        print()

    def growth_metrics_trends(self, historical_data, strain_name, current_starter_date):
        """Create run charts for growth metrics over time with current batch highlighted"""
        print("📈 GROWTH METRICS TRENDS")
        print("-" * 26)

        growth_metrics = {
            'TFI_minutes': 'Time to First Injection (min)',
            'Avg. IIT_minutes': 'Average Interinjection Time (min)',
            'Stdev IIT_minutes': 'Stdev Interinjection Time (min)',
            'Min IIT_minutes': 'Min Interinjection Time (min)',
            'Max IIT_minutes': 'Max Interinjection Time (min)',
            'pH CALI': 'pH Change After Last Injection'
        }

        # Sort by starter made date for chronological order
        historical_data = historical_data.sort_values('Starter_Made_Date')

        available_metrics = [col for col in growth_metrics.keys() if col in historical_data.columns]

        if available_metrics:
            fig, axes = plt.subplots(len(available_metrics), 1, figsize=(12, 3*len(available_metrics)))
            if len(available_metrics) == 1:
                axes = [axes]

            # Set white background
            fig.patch.set_facecolor('white')

            for i, metric_col in enumerate(available_metrics):
                axes[i].set_facecolor('white')
                data = historical_data[['Starter_Made_Date', metric_col]].dropna()

                if len(data) > 1:
                    axes[i].plot(data['Starter_Made_Date'], data[metric_col], 'o-', linewidth=2, markersize=4, color='navy')

                    # Highlight the current batch date with a special marker
                    current_batch_data = data[data['Starter_Made_Date'] == current_starter_date]
                    if not current_batch_data.empty:
                        axes[i].scatter(current_batch_data['Starter_Made_Date'],
                                      current_batch_data[metric_col],
                                      s=150, marker='*', color='red', zorder=10,
                                      edgecolor='darkred', linewidth=2,
                                      label='Current Batch')

                    axes[i].set_ylabel(growth_metrics[metric_col])

                    # Light gray grid lines in background
                    axes[i].grid(True, alpha=0.4, color='gray', linestyle='-', linewidth=0.5)
                    axes[i].set_axisbelow(True)
                    axes[i].tick_params(axis='x', rotation=45)

                    # Format x-axis dates as MM-DD
                    import matplotlib.dates as mdates
                    axes[i].xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))

                    # Add overall average line
                    overall_avg = data[metric_col].mean()
                    axes[i].axhline(y=overall_avg, color='red', linestyle='--', alpha=0.8,
                                  label=f'Overall Avg: {overall_avg:.2f}')
                    axes[i].legend(loc='upper right')
                else:
                    axes[i].text(0.5, 0.5, 'Insufficient data', ha='center', va='center', transform=axes[i].transAxes)
                    axes[i].set_ylabel(growth_metrics[metric_col])
                    axes[i].set_facecolor('white')
                    axes[i].grid(True, alpha=0.4, color='gray', linestyle='-', linewidth=0.5)
                    axes[i].set_axisbelow(True)

            plt.suptitle(f'Growth Metrics Trends for {strain_name}', y=0.99)
            plt.tight_layout()
            plt.show()
        else:
            print("  No growth metrics data available for plotting")
        print()

    def performance_by_intermediate(self, usage_data, strain_name, starter_date):
        """Analyze performance broken down by intermediate in table format"""
        if 'Intermediate' not in usage_data.columns:
            return

        print("🏭 PERFORMANCE BY INTERMEDIATE")
        print("-" * 35)

        intermediates = usage_data['Intermediate'].unique()

        if len(intermediates) == 1:
            print(f"  📦 Single intermediate: {intermediates[0]}")
            print(f"  📊 All {len(usage_data)} lots used same intermediate")
        else:
            # Create table format
            print(f"{'Intermediate':<40} {'Lots':<6} {'FG pH Avg':<10} {'FG pH Std':<10} {'FG Moist Avg':<12} {'FG Moist Std':<12}")
            print("-" * 102)

            for intermediate in sorted(intermediates):
                int_data = usage_data[usage_data['Intermediate'] == intermediate]

                # Clean up intermediate name
                if ' - ' in intermediate:
                    clean_name = intermediate.split(' - ', 1)[1]
                else:
                    clean_name = intermediate

                # Truncate if too long
                display_name = clean_name[:39] if len(clean_name) > 39 else clean_name

                lot_count = len(int_data)

                # FG pH statistics
                if 'FG pH' in int_data.columns:
                    ph_data = int_data['FG pH'][int_data['FG pH'] > 0]
                    if len(ph_data) > 0:
                        ph_avg = ph_data.mean()
                        ph_std = ph_data.std()
                        ph_avg_str = f"{ph_avg:.2f}"
                        ph_std_str = f"{ph_std:.2f}"
                    else:
                        ph_avg_str = "N/A"
                        ph_std_str = "N/A"
                else:
                    ph_avg_str = "N/A"
                    ph_std_str = "N/A"

                # FG Moisture statistics
                if 'FG Moisture' in int_data.columns:
                    moist_data = int_data['FG Moisture'][int_data['FG Moisture'] > 0]
                    if len(moist_data) > 0:
                        moist_avg = moist_data.mean()
                        moist_std = moist_data.std()
                        moist_avg_str = f"{moist_avg:.2f}%"
                        moist_std_str = f"{moist_std:.2f}%"
                    else:
                        moist_avg_str = "N/A"
                        moist_std_str = "N/A"
                else:
                    moist_avg_str = "N/A"
                    moist_std_str = "N/A"

                # Print table row
                print(f"{display_name:<40} {lot_count:<6} {ph_avg_str:<10} {ph_std_str:<10} {moist_avg_str:<12} {moist_std_str:<12}")
        print()

    def pH_performance_analysis(self, usage_data, strain_name, starter_date):
        """Create pH analysis across all usage of the starter batch"""
        print("🧪 pH PERFORMANCE ANALYSIS")
        print("-" * 30)

        ph_columns = ['pH to DMC', 'pH at First Turn', 'pH at Dice Mill', 'FG pH']

        # Filter out missing data (both NaN and zeros)
        ph_data = []
        labels = []
        for col in ph_columns:
            if col in usage_data.columns:
                data = usage_data[col].dropna()
                data = data[data > 0]  # Remove zeros
                if len(data) > 0:
                    ph_data.append(data)
                    labels.append(col)
                    print(f"  {col}: {len(data)} valid measurements (range: {data.min():.2f} - {data.max():.2f})")

        if ph_data:
            fig, ax = plt.subplots(figsize=(12, 8))

            # Set white background
            fig.patch.set_facecolor('white')
            ax.set_facecolor('white')

            # Create strip plot
            for i, data in enumerate(ph_data):
                # Add some random jitter for better visualization
                x_pos = [i + 1 + np.random.normal(0, 0.04) for _ in range(len(data))]
                ax.scatter(x_pos, data, alpha=0.6, s=50, color='navy', edgecolors='white', linewidth=0.5)

                # Add mean line
                mean_val = data.mean()
                ax.hlines(mean_val, i + 0.7, i + 1.3, colors='red', linestyles='-', linewidth=2, alpha=0.8)

                # Add median line (lighter)
                median_val = data.median()
                ax.hlines(median_val, i + 0.7, i + 1.3, colors='orange', linestyles='--', linewidth=1.5, alpha=0.7)

            ax.set_xticks(range(1, len(ph_data)+1))
            ax.set_xticklabels(labels, rotation=0)
            ax.set_title(f'pH Performance for {strain_name} Starter Made {starter_date.strftime("%m-%d")}')
            ax.set_ylabel('pH Value')

            # Light gray grid lines in background
            ax.grid(True, alpha=0.4, color='gray', linestyle='-', linewidth=0.5)
            ax.set_axisbelow(True)

            # Add legend
            from matplotlib.lines import Line2D
            legend_elements = [Line2D([0], [0], color='red', lw=2, label='Mean'),
                             Line2D([0], [0], color='orange', lw=1.5, linestyle='--', label='Median')]
            ax.legend(handles=legend_elements, loc='upper right')

            plt.tight_layout()
            plt.show()
        else:
            print("  No pH data available for plotting")
        print()

    def pH_distribution_analysis(self, usage_data, strain_name, starter_date):
        """Create finished goods pH distribution analysis across all usage of the starter batch"""
        print("🧪 FINISHED GOODS pH ANALYSIS")
        print("-" * 32)

        if 'FG pH' not in usage_data.columns:
            print("  ❌ 'FG pH' column not found in data")
            return

        # Get FG pH data for distribution
        fg_ph_data = usage_data['FG pH'].dropna()
        fg_ph_data = fg_ph_data[fg_ph_data > 0]  # Remove zeros

        if len(fg_ph_data) == 0:
            print("  No valid FG pH data available")
            return

        print(f"  📊 {len(fg_ph_data)} valid FG pH measurements")
        print(f"  📈 Range: {fg_ph_data.min():.2f} - {fg_ph_data.max():.2f}")
        print(f"  📈 Average: {fg_ph_data.mean():.2f} ± {fg_ph_data.std():.2f}")

        # Create side-by-side plots (distribution + by intermediate if available)
        if 'Intermediate' in usage_data.columns and len(usage_data['Intermediate'].unique()) > 1:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), gridspec_kw={'width_ratios': [1, 2]})
        else:
            fig, ax1 = plt.subplots(1, 1, figsize=(6, 6))
            ax2 = None

        # Set white background
        fig.patch.set_facecolor('white')
        ax1.set_facecolor('white')

        # Left plot: Strip plot distribution
        x_pos = [1 + np.random.normal(0, 0.02) for _ in range(len(fg_ph_data))]
        ax1.scatter(x_pos, fg_ph_data, alpha=0.6, s=50, color='navy', edgecolors='white', linewidth=0.5)

        # Add mean and median lines
        mean_val = fg_ph_data.mean()
        median_val = fg_ph_data.median()
        ax1.hlines(mean_val, 0.8, 1.2, colors='red', linestyles='-', linewidth=2, alpha=0.8)
        ax1.hlines(median_val, 0.8, 1.2, colors='orange', linestyles='--', linewidth=1.5, alpha=0.7)

        ax1.set_xticks([1])
        ax1.set_xticklabels(['FG pH'])
        ax1.set_title('FG pH Distribution')
        ax1.set_ylabel('pH Value')
        ax1.set_xlim(0.6, 1.4)

        # Grid and legend
        ax1.grid(True, alpha=0.4, color='gray', linestyle='-', linewidth=0.5)
        ax1.set_axisbelow(True)

        from matplotlib.lines import Line2D
        legend_elements = [Line2D([0], [0], color='red', lw=2, label='Mean'),
                         Line2D([0], [0], color='orange', lw=1.5, linestyle='--', label='Median')]
        ax1.legend(handles=legend_elements, loc='upper right')

        # Right plot: By intermediate (if multiple intermediates exist)
        if ax2 is not None:
            ax2.set_facecolor('white')
            intermediates = sorted(usage_data['Intermediate'].unique())

            int_data = []
            int_labels = []
            for intermediate in intermediates:
                int_fg_ph = usage_data[usage_data['Intermediate'] == intermediate]['FG pH']
                int_fg_ph = int_fg_ph.dropna()
                int_fg_ph = int_fg_ph[int_fg_ph > 0]
                if len(int_fg_ph) > 0:
                    int_data.append(int_fg_ph)
                    # Clean up label - remove IM Item No, keep only description
                    if ' - ' in intermediate:
                        desc_only = intermediate.split(' - ', 1)[1]
                    else:
                        desc_only = intermediate
                    int_labels.append(desc_only)

            if int_data:
                for i, data in enumerate(int_data):
                    x_pos = [i + 1 + np.random.normal(0, 0.02) for _ in range(len(data))]
                    ax2.scatter(x_pos, data, alpha=0.6, s=50, color='navy', edgecolors='white', linewidth=0.5)

                    # Add mean line for each intermediate
                    mean_val = data.mean()
                    ax2.hlines(mean_val, i + 0.8, i + 1.2, colors='red', linestyles='-', linewidth=2, alpha=0.8)

                    # Add mean value annotation next to the data cluster
                    ax2.text(i + 1.3, mean_val, f'{mean_val:.2f}', va='center', ha='left',
                            fontsize=10, color='black', weight='bold')

                ax2.set_xticks(range(1, len(int_data)+1))
                ax2.set_xticklabels(int_labels, rotation=0, ha='center')  # Horizontal labels
                ax2.set_title('FG pH by Intermediate')
                ax2.set_ylabel('pH Value')

                ax2.grid(True, alpha=0.4, color='gray', linestyle='-', linewidth=0.5)
                ax2.set_axisbelow(True)

        plt.tight_layout()
        plt.show()
        print()

    def moisture_performance_analysis(self, usage_data, strain_name, starter_date):
        """Create moisture analysis across all usage of the starter batch"""
        print("💧 MOISTURE PERFORMANCE ANALYSIS")
        print("-" * 35)

        if 'FG Moisture' not in usage_data.columns:
            print("  ❌ 'FG Moisture' column not found in data")
            return

        raw_moisture = usage_data['FG Moisture']
        moisture_data = raw_moisture.dropna()
        moisture_data = moisture_data[moisture_data > 0]  # Remove zeros

        if len(moisture_data) == 0:
            print("  No valid moisture data")
            return

        print(f"  📊 {len(moisture_data)} valid moisture measurements")
        print(f"  📈 Range: {moisture_data.min():.2f} - {moisture_data.max():.2f}%")
        print(f"  📈 Average: {moisture_data.mean():.2f}% ± {moisture_data.std():.2f}%")

        # Create side-by-side plots (distribution + by intermediate if available)
        if 'Intermediate' in usage_data.columns and len(usage_data['Intermediate'].unique()) > 1:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), gridspec_kw={'width_ratios': [1, 2]})
        else:
            fig, ax1 = plt.subplots(1, 1, figsize=(6, 6))
            ax2 = None

        # Set white background
        fig.patch.set_facecolor('white')
        ax1.set_facecolor('white')

        # Left plot: Strip plot distribution
        x_pos = [1 + np.random.normal(0, 0.02) for _ in range(len(moisture_data))]
        ax1.scatter(x_pos, moisture_data, alpha=0.6, s=50, color='darkgreen', edgecolors='white', linewidth=0.5)

        # Add mean and median lines
        mean_val = moisture_data.mean()
        median_val = moisture_data.median()
        ax1.hlines(mean_val, 0.8, 1.2, colors='red', linestyles='-', linewidth=2, alpha=0.8)
        ax1.hlines(median_val, 0.8, 1.2, colors='orange', linestyles='--', linewidth=1.5, alpha=0.7)

        ax1.set_xticks([1])
        ax1.set_xticklabels(['Moisture'])
        ax1.set_title('Moisture Distribution')
        ax1.set_ylabel('Moisture (%)')
        ax1.set_xlim(0.6, 1.4)

        # Grid and legend
        ax1.grid(True, alpha=0.4, color='gray', linestyle='-', linewidth=0.5)
        ax1.set_axisbelow(True)

        from matplotlib.lines import Line2D
        legend_elements = [Line2D([0], [0], color='red', lw=2, label='Mean'),
                         Line2D([0], [0], color='orange', lw=1.5, linestyle='--', label='Median')]
        ax1.legend(handles=legend_elements, loc='upper right')

        # Right plot: By intermediate (if multiple intermediates exist)
        if ax2 is not None:
            ax2.set_facecolor('white')
            intermediates = sorted(usage_data['Intermediate'].unique())

            int_data = []
            int_labels = []
            for intermediate in intermediates:
                int_moisture = usage_data[usage_data['Intermediate'] == intermediate]['FG Moisture']
                int_moisture = int_moisture.dropna()
                int_moisture = int_moisture[int_moisture > 0]
                if len(int_moisture) > 0:
                    int_data.append(int_moisture)
                    # Clean up label - remove IM Item No and (n=x), keep only description
                    if ' - ' in intermediate:
                        desc_only = intermediate.split(' - ', 1)[1]  # Take everything after first " - "
                    else:
                        desc_only = intermediate
                    int_labels.append(desc_only)

            if int_data:
                for i, data in enumerate(int_data):
                    x_pos = [i + 1 + np.random.normal(0, 0.02) for _ in range(len(data))]
                    ax2.scatter(x_pos, data, alpha=0.6, s=50, color='darkgreen', edgecolors='white', linewidth=0.5)

                    # Add mean line for each intermediate
                    mean_val = data.mean()
                    ax2.hlines(mean_val, i + 0.8, i + 1.2, colors='red', linestyles='-', linewidth=2, alpha=0.8)

                    # Add mean value annotation next to the data cluster
                    ax2.text(i + 1.3, mean_val, f'{mean_val:.2f}%', va='center', ha='left',
                            fontsize=10, color='black', weight='bold')

                ax2.set_xticks(range(1, len(int_data)+1))
                ax2.set_xticklabels(int_labels, rotation=0, ha='center')  # Horizontal labels
                ax2.set_title('Moisture by Intermediate')
                ax2.set_ylabel('Moisture (%)')

                ax2.grid(True, alpha=0.4, color='gray', linestyle='-', linewidth=0.5)
                ax2.set_axisbelow(True)

        plt.tight_layout()
        plt.show()
        print()

    def process_metrics_by_vat(self, usage_data, strain_name, starter_date):
        """Create charts showing process metrics performance by vat with enhanced intermediate handling"""
        print("🏭 PROCESS METRICS BY VAT")
        print("-" * 28)

        # Extract vat number from Lot Number (last 2 digits)
        def extract_vat_from_lot(lot_number):
            try:
                lot_str = str(int(lot_number))  # Convert to string, removing any decimals
                if len(lot_str) >= 2:
                    return int(lot_str[-2:])  # Last 2 digits
                else:
                    return None
            except:
                return None

        vat_data = usage_data.copy()
        vat_data['Vat'] = vat_data['Lot Number'].apply(extract_vat_from_lot)
        vat_data = vat_data.dropna(subset=['Vat'])

        if vat_data.empty:
            print("  Could not extract vat numbers from lot numbers")
            return

        # Smart ordering for cross-day production with date tracking
        # Sort vats intelligently to handle cross-day sequences like [46,47,48,49,50,51,1,2,3...]
        sorted_vats = sorted(vat_data['Vat'].unique())

        # Check if we have a cross-day pattern (high vats followed by low vats)
        cross_day_detected = False
        vat_labels = []  # Will store vat labels with dates if needed

        if len(sorted_vats) > 1:
            # Look for gaps that suggest day transition
            gaps = []
            for i in range(1, len(sorted_vats)):
                gap = sorted_vats[i] - sorted_vats[i-1]
                gaps.append((gap, i, sorted_vats[i-1], sorted_vats[i]))

            # Find largest negative gap (suggests day rollover)
            largest_negative_gap = None
            for gap, idx, prev_vat, curr_vat in gaps:
                if gap < -20:  # Significant negative jump (e.g., 51 -> 1)
                    if largest_negative_gap is None or gap < largest_negative_gap[0]:
                        largest_negative_gap = (gap, idx, prev_vat, curr_vat)

            # If we found a day transition, reorder the vats chronologically
            if largest_negative_gap:
                split_idx = largest_negative_gap[1]
                # Put the later vats (after transition) first, then early vats
                reordered_vats = sorted_vats[split_idx:] + sorted_vats[:split_idx]

                # Create a custom sort order
                vat_order = {vat: idx for idx, vat in enumerate(reordered_vats)}
                vat_data['Sort_Order'] = vat_data['Vat'].map(vat_order)
                vat_data = vat_data.sort_values('Sort_Order')
                cross_day_detected = True

                # Create vat labels with dates for cross-day production
                for i, vat in enumerate(reordered_vats):
                    vat_row = vat_data[vat_data['Vat'] == vat].iloc[0]
                    if 'DOM' in vat_row and pd.notna(vat_row['DOM']):
                        date_str = pd.to_datetime(vat_row['DOM']).strftime('%m/%d')
                        vat_labels.append(f"{vat}\n{date_str}")
                    else:
                        vat_labels.append(str(vat))
            else:
                # Normal sorting by vat number
                vat_data = vat_data.sort_values('Vat')
                vat_labels = [str(vat) for vat in sorted_vats]
        else:
            vat_data = vat_data.sort_values('Vat')
            vat_labels = [str(vat) for vat in sorted_vats]

        process_metrics = {
            'Starter QTY': 'Starter Quantity',
            'pH to DMC': 'pH to DMC',
            'pH at First Turn': 'pH at First Turn',
            'pH at Dice Mill': 'pH at Dice Mill',
            'FG pH': 'Finished Goods pH'
        }

        available_metrics = [col for col in process_metrics.keys() if col in vat_data.columns]

        if available_metrics:
            fig, axes = plt.subplots(len(available_metrics), 1, figsize=(12, 3*len(available_metrics)))
            if len(available_metrics) == 1:
                axes = [axes]

            # Set white background
            fig.patch.set_facecolor('white')

            # Determine intermediate groupings based on Starter QTY (most complete field)
            intermediate_ranges = {}
            if 'Starter QTY' in vat_data.columns:
                # Use Starter QTY to identify intermediate groups since it has fewer missing values
                starter_qty_data = vat_data[['Vat', 'Starter QTY', 'Intermediate']].dropna(subset=['Starter QTY'])

                if not starter_qty_data.empty:
                    for _, row in starter_qty_data.iterrows():
                        intermediate = row['Intermediate']
                        vat = row['Vat']

                        # Clean up intermediate name
                        if ' - ' in intermediate:
                            clean_name = intermediate.split(' - ', 1)[1]
                        else:
                            clean_name = intermediate

                        if clean_name not in intermediate_ranges:
                            intermediate_ranges[clean_name] = []
                        intermediate_ranges[clean_name].append(vat)

                    # Sort vat ranges within each intermediate
                    for intermediate in intermediate_ranges:
                        intermediate_ranges[intermediate] = sorted(list(set(intermediate_ranges[intermediate])))

            for i, metric_col in enumerate(available_metrics):
                axes[i].set_facecolor('white')

                # For pH columns, filter out zeros
                plot_data = vat_data[['Vat', metric_col, 'Intermediate']].copy()
                if 'pH' in metric_col:
                    plot_data = plot_data[plot_data[metric_col] > 0]

                plot_data = plot_data.dropna()

                if len(plot_data) > 0:
                    axes[i].plot(plot_data['Vat'], plot_data[metric_col], 'o-', linewidth=2, markersize=6, color='navy')
                    axes[i].set_ylabel(process_metrics[metric_col])

                    # Light gray grid lines in background
                    axes[i].grid(True, alpha=0.4, color='gray', linestyle='-', linewidth=0.5)
                    axes[i].set_axisbelow(True)

                    # Set x-axis to show all vat numbers with proper labels
                    if len(plot_data) > 1:
                        vat_list = plot_data['Vat'].tolist()
                        axes[i].set_xticks(vat_list)

                        # Use date labels if cross-day detected, otherwise just vat numbers
                        if cross_day_detected:
                            display_labels = []
                            for vat in vat_list:
                                vat_row = vat_data[vat_data['Vat'] == vat].iloc[0]
                                if 'DOM' in vat_row and pd.notna(vat_row['DOM']):
                                    date_str = pd.to_datetime(vat_row['DOM']).strftime('%m/%d')
                                    display_labels.append(f"{vat}\n{date_str}")
                                else:
                                    display_labels.append(str(vat))
                            axes[i].set_xticklabels(display_labels, fontsize=8)
                        else:
                            axes[i].set_xticklabels([str(vat) for vat in vat_list])

                    # Add intermediate-specific reference lines (ALL RED)
                    if intermediate_ranges and 'pH' in metric_col:
                        for intermediate, vats in intermediate_ranges.items():
                            # Get data for this intermediate
                            int_data = plot_data[plot_data['Vat'].isin(vats)][metric_col]

                            if len(int_data) > 1:
                                int_avg = int_data.mean()

                                # Draw reference line only for vats in this intermediate (ALL RED)
                                min_vat = min([v for v in plot_data['Vat'] if v in vats])
                                max_vat = max([v for v in plot_data['Vat'] if v in vats])

                                axes[i].hlines(int_avg, min_vat, max_vat,
                                             colors='red', linestyles='--', alpha=0.8, linewidth=2)

                                # Add small data label next to reference line (RED)
                                label_x = min_vat + (max_vat - min_vat) * 0.15
                                axes[i].text(label_x, int_avg, f'{int_avg:.2f}',
                                           fontsize=8, color='red', weight='bold',
                                           ha='left', va='bottom',
                                           bbox=dict(boxstyle='round,pad=0.2', facecolor='white',
                                                   edgecolor='red', alpha=0.8))
                    elif not 'pH' in metric_col:
                        # For non-pH metrics (like Starter QTY), add overall average (RED)
                        if len(plot_data) > 1:
                            avg_value = plot_data[metric_col].mean()
                            axes[i].axhline(y=avg_value, color='red', linestyle='--', alpha=0.8)
                            axes[i].text(plot_data['Vat'].iloc[0], avg_value, f'Avg: {avg_value:.2f}',
                                       fontsize=8, color='red', weight='bold', ha='left', va='bottom',
                                       bbox=dict(boxstyle='round,pad=0.2', facecolor='white',
                                               edgecolor='red', alpha=0.8))

                    # Add intermediate background shading
                    if intermediate_ranges:
                        bg_colors = ['blue', 'green', 'orange', 'purple', 'brown']
                        color_idx = 0

                        for intermediate, vats in intermediate_ranges.items():
                            if len(vats) > 0:
                                min_vat = min(vats)
                                max_vat = max(vats)
                                bg_color = bg_colors[color_idx % len(bg_colors)]

                                # Add colored background shading for this intermediate
                                axes[i].axvspan(min_vat - 0.4, max_vat + 0.4, alpha=0.1,
                                              color=bg_color, zorder=0)
                                color_idx += 1
                else:
                    axes[i].text(0.5, 0.5, 'No valid data', ha='center', va='center',
                               transform=axes[i].transAxes)
                    axes[i].set_ylabel(process_metrics[metric_col])
                    axes[i].set_facecolor('white')
                    axes[i].grid(True, alpha=0.4, color='gray', linestyle='-', linewidth=0.5)
                    axes[i].set_axisbelow(True)

                # Set xlabel only on bottom chart
                if i == len(available_metrics) - 1:
                    if cross_day_detected:
                        axes[i].set_xlabel('Vat Number\n(with Production Date)')
                    else:
                        axes[i].set_xlabel('Vat Number')

            # Add intermediate labels ABOVE the entire chart area (outside plot area)
            if intermediate_ranges:
                bg_colors = ['blue', 'green', 'orange', 'purple', 'brown']
                color_idx = 0

                # Use the top axis to add lines above the chart area
                top_ax = axes[0]  # Use the top chart for positioning

                # Get current y-limits to position labels above
                y_min, y_max = top_ax.get_ylim()
                y_range = y_max - y_min
                label_line_y = y_max + y_range * 0.23  # Position for the line
                label_text_y = y_max + y_range * 0.30  # Position for the text

                for intermediate, vats in intermediate_ranges.items():
                    if len(vats) > 0:
                        min_vat = min(vats)
                        max_vat = max(vats)
                        center_vat = (min_vat + max_vat) / 2
                        bg_color = bg_colors[color_idx % len(bg_colors)]

                        # Add colored line that spans exactly the vat range
                        top_ax.plot([min_vat, max_vat], [label_line_y, label_line_y],
                                  color=bg_color, linewidth=4, solid_capstyle='butt')

                        # Add intermediate name above the line
                        top_ax.text(center_vat, label_text_y, intermediate,
                                  ha='center', va='bottom', fontsize=10,
                                  color=bg_color, weight='bold')

                        color_idx += 1

                # Extend the y-axis of the top chart to accommodate labels
                top_ax.set_ylim(y_min, y_max + y_range * 0.25)

            plt.suptitle(f'Process Metrics by Vat for {strain_name} Starter Made {starter_date.strftime("%m-%d")}', y=0.99)
            plt.tight_layout(rect=[0, 0.03, 1, 0.99])
            plt.show()

            print(f"  📊 Analyzed {len(vat_data)} lots across {vat_data['Vat'].nunique()} vats")
            if cross_day_detected:
                print(f"  🔄 Detected cross-day production pattern - vats reordered chronologically with dates")
        else:
            print("  No process metrics data available for plotting")
        print()

# Usage function
def create_batch_analyzer(file_path="/content/drive/MyDrive/FFUSA/RC Starter Tank Monitoring/26-02-starter-analysis.xlsx", debug_batch_ids=False):
    """
    Create and display the starter batch analyzer
    """
    try:
        print("🚀 Loading Starter Batch Analyzer...")
        analyzer = StarterBatchAnalyzer(file_path)

        print(f"✅ Data loaded and processed successfully!")
        print(f"   📊 Total records: {len(analyzer.df)}")

        # Check if we have valid dates and automatically debug if not
        if 'Starter_Made_Date' in analyzer.df.columns:
            valid_dates = analyzer.df['Starter_Made_Date'].dropna()
            datetime_dates = [d for d in valid_dates if pd.notna(d) and isinstance(d, (pd.Timestamp, datetime))]

            if len(datetime_dates) > 0:
                print(f"   📅 Starter date range: {min(datetime_dates).strftime('%Y-%m-%d')} to {max(datetime_dates).strftime('%Y-%m-%d')}")
            else:
                print("   ⚠️ No valid starter dates found - running automatic debug...")
                debug_batch_ids = True

        # Debug batch IDs if requested or if there are issues
        if debug_batch_ids:
            analyzer.debug_batch_ids()

        print(f"   🧪 Unique strains: {analyzer.df['Starter Strain'].nunique()}")
        if 'Intermediate' in analyzer.df.columns:
            print(f"   🏭 Unique intermediates: {analyzer.df['Intermediate'].nunique()}")
        print()

        # Display the interactive interface
        analyzer.display_interface()

        return analyzer

    except FileNotFoundError:
        print("❌ File not found. Please check your file path.")
        print("Make sure you've mounted Google Drive and the file path is correct.")
        return None
    except Exception as e:
        print(f"❌ Error: {e}")
        print("\n🔍 Running diagnostic debug...")
        try:
            # Try to load the file and run debug
            analyzer = StarterBatchAnalyzer(file_path)
            analyzer.debug_batch_ids()
        except Exception as debug_e:
            print(f"Debug also failed: {debug_e}")
            import traceback
            print("Full error details:")
            traceback.print_exc()
        return None

# Example usage
if __name__ == "__main__":
    # Create the batch analyzer
    analyzer = create_batch_analyzer()

🚀 Loading Starter Batch Analyzer...
✅ Data loaded and processed successfully!
   📊 Total records: 1227
   📅 Starter date range: 2026-01-09 to 2026-02-02
   🧪 Unique strains: 10
   🏭 Unique intermediates: 11

🧪 STARTER BATCH PERFORMANCE ANALYZER
🎯 New Analysis Focus:
1. Select when the starter was MADE
2. Choose which strain was made that day
3. Analyze performance across ALL intermediates using that starter


In [40]:
"""
Starter Batch Analysis System - Restructured for Batch-Centric Analysis
Author: Process Improvement Team
Description: Interactive analysis focused on starter batches and their performance across intermediates
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import widgets for interactivity
import ipywidgets as widgets
from IPython.display import display, clear_output

# Set up plotting style
plt.style.use('default')
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.4
plt.rcParams['grid.color'] = 'gray'
sns.set_palette("husl")

class StarterBatchAnalyzer:
    def __init__(self, file_path, recipe_file_path=None):
        """Initialize the analyzer with the main data and optional recipe specifications"""
        self.df = pd.read_excel(file_path)
        self.recipe_specs = None

        # Load recipe specifications if provided
        if recipe_file_path:
            try:
                self.recipe_specs = pd.read_excel(recipe_file_path)
                print(f"✅ Recipe specifications loaded: {len(self.recipe_specs)} intermediates")
            except Exception as e:
                print(f"⚠️  Could not load recipe specifications: {e}")
                self.recipe_specs = None

        self.prepare_data()
        self.setup_widgets()

    def prepare_data(self):
        """Clean and prepare data for batch-centric analysis"""
        # Debug: Print available columns
        print(f"🔍 Available columns in data file:")
        print(f"   {list(self.df.columns)}")

        # Convert date columns
        self.df['DOM'] = pd.to_datetime(self.df['DOM'])

        # Extract starter batch information from Starter Batch ID
        self.df['Starter_Made_Date'] = self.df['Starter Batch ID'].apply(self.extract_starter_date)
        self.df['Starter_Tank'] = self.df['Starter Batch ID'].apply(self.extract_starter_tank)
        self.df['Strain_Number'] = self.df['Starter Batch ID'].apply(self.extract_strain_number)

        # Convert time columns to minutes for easier analysis
        # Check for both versions (with and without periods) since your data has periods
        time_cols_mapping = {
            'TFI': 'TFI_minutes',
            'Avg. IIT': 'Avg. IIT_minutes',
            'Avg IIT': 'Avg. IIT_minutes',  # Alternative without period
            'Stdev IIT': 'Stdev IIT_minutes',
            'Stdev. IIT': 'Stdev IIT_minutes',  # Alternative with period
            'Min IIT': 'Min IIT_minutes',
            'Min. IIT': 'Min IIT_minutes',  # Alternative with period
            'Max IIT': 'Max IIT_minutes',
            'Max. IIT': 'Max IIT_minutes'   # Alternative with period
        }

        for col, new_col in time_cols_mapping.items():
            if col in self.df.columns:
                self.df[new_col] = self.df[col].apply(self.time_to_minutes)

        # Remove rows with missing critical data
        self.df = self.df.dropna(subset=['DOM', 'Starter Strain', 'Starter Batch ID'])

        # Convert strain names to strings for consistent handling
        self.df['Starter Strain'] = self.df['Starter Strain'].astype(str)

        # Create intermediate identifier if columns exist - FIXED to handle periods
        intermediate_created = False
        possible_item_cols = ['IM Item No', 'IM Item No.', 'IM_Item_No', 'im_item_no', 'Item No', 'Item No.', 'Item_No']
        possible_desc_cols = ['IM Item Desc', 'IM Item Desc.', 'IM_Item_Desc', 'im_item_desc', 'Item Desc', 'Item Desc.', 'Item_Desc', 'Description']

        item_col = None
        desc_col = None

        # Find the correct column names
        for col in possible_item_cols:
            if col in self.df.columns:
                item_col = col
                break

        for col in possible_desc_cols:
            if col in self.df.columns:
                desc_col = col
                break

        if item_col and desc_col:
            self.df['Intermediate'] = self.df[item_col].astype(str) + ' - ' + self.df[desc_col].astype(str)
            intermediate_created = True
            print(f"✅ Intermediate column created using '{item_col}' and '{desc_col}'")
        else:
            self.df['Intermediate'] = 'Unknown Intermediate'
            print(f"⚠️  Could not find IM Item columns.")
            print(f"   Looking for item: {possible_item_cols}")
            print(f"   Looking for desc: {possible_desc_cols}")
            print(f"   Found columns: {list(self.df.columns)}")

        # Show sample intermediate values
        if intermediate_created:
            print(f"   Sample intermediates: {list(self.df['Intermediate'].unique()[:5])}")

        # Merge recipe specifications if available - FIXED to avoid row duplication
        if self.recipe_specs is not None and item_col:
            # Print recipe specs columns for debugging
            print(f"🔍 Recipe specs columns: {list(self.recipe_specs.columns)}")

            # Show record counts before merge
            original_count = len(self.df)
            print(f"📊 Original data records: {original_count}")
            print(f"📊 Recipe spec records: {len(self.recipe_specs)}")

            # Create a clean merge dataframe with just the columns we need
            merge_df = self.recipe_specs[['im_item_no', 'im_moi_max', 'im_moi_min', 'im_ph_max', 'im_ph_min']].copy()

            # Remove any duplicate item numbers from recipe specs to prevent row multiplication
            original_spec_count = len(merge_df)
            merge_df = merge_df.drop_duplicates(subset=['im_item_no'])
            dedupe_spec_count = len(merge_df)

            if original_spec_count != dedupe_spec_count:
                print(f"   ⚠️  Removed {original_spec_count - dedupe_spec_count} duplicate recipe specs")

            # Rename columns for the merge
            merge_df = merge_df.rename(columns={
                'im_item_no': 'merge_item_no',
                'im_moi_max': 'Moisture_Max_Spec',
                'im_moi_min': 'Moisture_Min_Spec',
                'im_ph_max': 'pH_Max_Spec',
                'im_ph_min': 'pH_Min_Spec'
            })

            # Create standardized merge key in main dataframe
            self.df['merge_item_no'] = self.df[item_col].astype(str)
            merge_df['merge_item_no'] = merge_df['merge_item_no'].astype(str)

            # Show merge key comparison for debugging
            main_keys = set(self.df['merge_item_no'].unique())
            spec_keys = set(merge_df['merge_item_no'].unique())
            print(f"📊 Main data unique item numbers: {len(main_keys)}")
            print(f"📊 Recipe spec unique item numbers: {len(spec_keys)}")
            print(f"📊 Matching item numbers: {len(main_keys.intersection(spec_keys))}")

            # Perform LEFT JOIN to avoid duplicating rows
            self.df = self.df.merge(merge_df, on='merge_item_no', how='left')

            # Clean up temporary merge column
            self.df = self.df.drop('merge_item_no', axis=1)

            # Check results
            final_count = len(self.df)
            specs_count = self.df[['Moisture_Max_Spec', 'pH_Max_Spec']].notna().any(axis=1).sum()

            print(f"✅ Recipe specifications merged successfully")
            print(f"   📊 Final record count: {final_count} (should equal {original_count})")
            print(f"   📊 Records with specification data: {specs_count}")

            if final_count != original_count:
                print(f"   ⚠️  WARNING: Record count changed from {original_count} to {final_count}")
                print(f"   This suggests duplicate matches in the recipe specs data")
        elif self.recipe_specs is not None:
            print(f"⚠️  Cannot merge recipe specs - no item column found")

    def extract_starter_date(self, batch_id):
        """Extract starter made date from Batch ID
        Format: 1072601091504 = 107 + 260109 + 15 + 04
        Positions 3-8 (YYMMDD): 260109 = 2026-01-09"""
        try:
            if pd.isna(batch_id):
                return None
            batch_str = str(int(float(batch_id)))  # Handle potential float values
            if len(batch_str) >= 9:  # Need at least 9 digits to extract date
                year = int('20' + batch_str[3:5])  # Positions 3-4: YY -> 20YY
                month = int(batch_str[5:7])        # Positions 5-6: MM
                day = int(batch_str[7:9])          # Positions 7-8: DD
                # Validate date components
                if 1 <= month <= 12 and 1 <= day <= 31 and 2020 <= year <= 2030:
                    return pd.to_datetime(f'{year}-{month:02d}-{day:02d}')
            return None
        except (ValueError, TypeError, IndexError):
            return None

    def extract_starter_tank(self, batch_id):
        """Extract starter tank from Batch ID (positions 9-10)"""
        try:
            if pd.isna(batch_id):
                return None
            batch_str = str(int(float(batch_id)))  # Handle potential float values
            if len(batch_str) >= 11:
                return int(batch_str[9:11])  # Positions 9-10
            return None
        except (ValueError, TypeError, IndexError):
            return None

    def extract_strain_number(self, batch_id):
        """Extract strain number from Batch ID (positions 11-12)"""
        try:
            if pd.isna(batch_id):
                return None
            batch_str = str(int(float(batch_id)))  # Handle potential float values
            if len(batch_str) >= 13:
                return int(batch_str[11:13])  # Positions 11-12
            return None
        except (ValueError, TypeError, IndexError):
            return None

    def time_to_minutes(self, time_val):
        """Convert time format to minutes"""
        if pd.isna(time_val):
            return np.nan
        try:
            # Handle string format like "0:06:54"
            if isinstance(time_val, str):
                time_val = time_val.strip()
                parts = time_val.split(':')
                if len(parts) == 3:
                    hours = int(parts[0])
                    minutes = int(parts[1])
                    seconds = int(parts[2])
                    return hours * 60 + minutes + seconds / 60
                else:
                    return np.nan
            # Handle datetime objects
            elif isinstance(time_val, datetime):
                return time_val.hour * 60 + time_val.minute + time_val.second / 60
            # Handle pandas Timedelta objects
            elif hasattr(time_val, 'total_seconds'):
                return time_val.total_seconds() / 60
            else:
                # Try to convert to string and parse
                time_str = str(time_val)
                if ':' in time_str:
                    parts = time_str.split(':')
                    if len(parts) >= 2:
                        hours = int(parts[0]) if parts[0].isdigit() else 0
                        minutes = int(parts[1]) if parts[1].isdigit() else 0
                        seconds = int(parts[2]) if len(parts) > 2 and parts[2].isdigit() else 0
                        return hours * 60 + minutes + seconds / 60
                return np.nan
        except (ValueError, IndexError, AttributeError) as e:
            print(f"Warning: Could not parse time value '{time_val}': {e}")
            return np.nan

    def debug_batch_ids(self):
        """Debug function to check Batch ID parsing"""
        print("🔍 BATCH ID PARSING DEBUG")
        print("-" * 35)

        print("📊 Sample Batch IDs and parsing results:")
        sample_batch_ids = self.df['Starter Batch ID'].dropna().head(10)

        for i, batch_id in enumerate(sample_batch_ids, 1):
            try:
                batch_str = str(int(float(batch_id)))
                date_extracted = self.extract_starter_date(batch_id)
                tank_extracted = self.extract_starter_tank(batch_id)
                strain_extracted = self.extract_strain_number(batch_id)

                print(f"  {i:2d}. Batch ID: {batch_id}")
                print(f"      → String: '{batch_str}' (length: {len(batch_str)})")
                print(f"      → Date: {date_extracted}")
                print(f"      → Tank: {tank_extracted}")
                print(f"      → Strain: {strain_extracted}")
                print()
            except Exception as e:
                print(f"  {i:2d}. Batch ID: {batch_id} → ERROR: {e}")
                print()

        # Summary stats
        total_ids = len(self.df['Starter Batch ID'].dropna())
        valid_dates = len(self.df['Starter_Made_Date'].dropna())

        print(f"📈 SUMMARY:")
        print(f"  Total Batch IDs: {total_ids}")
        print(f"  Valid dates extracted: {valid_dates}")
        print(f"  Success rate: {valid_dates/total_ids*100:.1f}%" if total_ids > 0 else "  Success rate: 0%")
        print()

    def setup_widgets(self):
        """Setup interactive widgets for starter batch selection"""
        # Get available starter made dates - filter out None values and ensure datetime type
        starter_dates = self.df['Starter_Made_Date'].dropna()

        # Additional check to ensure we have datetime objects
        valid_datetime_dates = []
        for date in starter_dates:
            if pd.notna(date) and isinstance(date, (pd.Timestamp, datetime)):
                valid_datetime_dates.append(date)

        if len(valid_datetime_dates) == 0:
            print("⚠️ Warning: No valid starter dates found in Batch IDs")
            print("Check that Starter Batch ID column contains valid batch numbers")
            self._create_empty_widgets()
            return

        # Convert to pandas Series for proper handling
        valid_dates_series = pd.Series(valid_datetime_dates)
        available_dates = sorted(valid_dates_series.dt.date.unique())

        # Date selection widget
        self.date_widget = widgets.Dropdown(
            options=[(date.strftime('%Y-%m-%d (%A)'), date) for date in available_dates],
            value=available_dates[0] if available_dates else None,
            description='Starter Made:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )

        # Strain selection widget (initially empty)
        self.strain_widget = widgets.Dropdown(
            options=[],
            description='Select Strain:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )

        # Generate report button
        self.generate_button = widgets.Button(
            description='🧪 Generate Batch Report',
            button_style='primary',
            layout=widgets.Layout(width='250px', height='40px')
        )

        # Output area for the report
        self.output = widgets.Output()

        # Set up event handlers
        self.date_widget.observe(self.on_date_change, names='value')
        self.generate_button.on_click(self.on_generate_click)

        # Initialize strain options for the first date
        if available_dates:
            self.update_strain_options()

    def _create_empty_widgets(self):
        """Create empty widgets when no valid data is found"""
        self.date_widget = widgets.Dropdown(
            options=[],
            description='Starter Made:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        self.strain_widget = widgets.Dropdown(
            options=[],
            description='Select Strain:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        self.generate_button = widgets.Button(
            description='🧪 Generate Batch Report',
            button_style='primary',
            layout=widgets.Layout(width='250px', height='40px')
        )
        self.output = widgets.Output()

    def on_date_change(self, change):
        """Handle starter made date selection change"""
        self.update_strain_options()

    def update_strain_options(self):
        """Update strain options based on selected starter made date"""
        if not self.date_widget.value:
            return

        selected_date = pd.to_datetime(self.date_widget.value)

        # Get strains that had starter made on the selected date
        date_batches = self.df[self.df['Starter_Made_Date'] == selected_date]
        available_strains = sorted(date_batches['Starter Strain'].unique())

        # Update strain widget options
        self.strain_widget.options = [(strain, strain) for strain in available_strains]

        # Set default value if options exist
        if available_strains:
            self.strain_widget.value = available_strains[0]

        # Clear previous output and show available info
        with self.output:
            clear_output(wait=True)
            if available_strains:
                print(f"📅 Starter batches made on {selected_date.strftime('%Y-%m-%d')}:")
                batch_info = date_batches[['Starter Strain', 'Starter_Tank', 'Starter Batch ID']].drop_duplicates()
                for _, row in batch_info.iterrows():
                    print(f"   🧪 Strain {row['Starter Strain']} - Tank {row['Starter_Tank']} - Batch {row['Starter Batch ID']}")
                print(f"\n💭 {len(available_strains)} strain(s) available for analysis")
            else:
                print(f"⚠️  No starter batches found for {selected_date.strftime('%Y-%m-%d')}")

    def on_generate_click(self, button):
        """Handle generate report button click"""
        with self.output:
            clear_output(wait=True)

            if not self.strain_widget.options:
                print("❌ No strain selected. Please choose a date with available starter batches.")
                return

            # Get selected values
            starter_made_date = pd.to_datetime(self.date_widget.value)
            strain_name = self.strain_widget.value

            # Generate the report
            self.generate_batch_report(starter_made_date, strain_name)

    def display_interface(self):
        """Display the interactive interface"""
        print("🧪 STARTER BATCH PERFORMANCE ANALYZER")
        print("="*60)
        print("🎯 New Analysis Focus:")
        print("1. Select when the starter was MADE")
        print("2. Choose which strain was made that day")
        print("3. Analyze performance across ALL intermediates using that starter")
        print("="*60)

        # Create interface layout
        controls = widgets.VBox([
            widgets.HBox([self.date_widget, self.strain_widget]),
            widgets.HBox([self.generate_button]),
            self.output
        ])

        display(controls)

    def generate_batch_report(self, starter_made_date, strain_name):
        """Generate comprehensive report for starter batch performance"""
        print(f"🧪 STARTER BATCH PERFORMANCE REPORT")
        print(f"{'='*70}")
        print(f"Starter Made Date: {starter_made_date.strftime('%Y-%m-%d (%A)')}")
        print(f"Strain: {strain_name}")
        print(f"Report Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"{'='*70}\n")

        # Get the specific starter batch(es) made on this date
        batch_info = self.df[
            (self.df['Starter_Made_Date'] == starter_made_date) &
            (self.df['Starter Strain'] == strain_name)
        ]

        if batch_info.empty:
            print(f"❌ No starter batches found for strain '{strain_name}' made on {starter_made_date.strftime('%Y-%m-%d')}")
            return

        # Get unique batch IDs for this strain/date combination
        batch_ids = batch_info['Starter Batch ID'].unique()

        # Get all lots that used any of these starter batches
        all_usage_data = self.df[self.df['Starter Batch ID'].isin(batch_ids)]

        print(f"📊 BATCH OVERVIEW")
        print("-" * 20)
        for batch_id in batch_ids:
            batch_specific = all_usage_data[all_usage_data['Starter Batch ID'] == batch_id]
            tank = batch_specific['Starter_Tank'].iloc[0] if len(batch_specific) > 0 else 'Unknown'
            usage_count = len(batch_specific)
            date_range = f"{batch_specific['DOM'].min().strftime('%m-%d')} to {batch_specific['DOM'].max().strftime('%m-%d')}" if usage_count > 1 else batch_specific['DOM'].iloc[0].strftime('%m-%d')
            print(f"  🆔 Batch {batch_id} (Tank {tank}): Used in {usage_count} lots from {date_range}")

        # Show intermediate breakdown
        if 'Intermediate' in all_usage_data.columns:
            intermediate_summary = all_usage_data.groupby('Intermediate').size().sort_values(ascending=False)
            print(f"\n🏭 INTERMEDIATE USAGE")
            print("-" * 20)
            for intermediate, count in intermediate_summary.items():
                print(f"  📦 {intermediate}: {count} lots")

        print(f"\n📈 Total lots analyzed: {len(all_usage_data)}")
        print(f"🗓️  Production date range: {all_usage_data['DOM'].min().strftime('%Y-%m-%d')} to {all_usage_data['DOM'].max().strftime('%Y-%m-%d')}")
        print()

        # Generate growth metrics (from the batch creation)
        growth_data = batch_info.iloc[0]  # Take first batch record for growth metrics
        self.growth_metrics_summary(growth_data)

        # Get historical data for this strain for comparison
        strain_historical = self.df[self.df['Starter Strain'] == strain_name]
        self.historical_comparison(growth_data, strain_historical)

        # Add growth metrics trends over time for this strain
        self.growth_metrics_trends(strain_historical, strain_name, starter_made_date)

        # Analyze performance across intermediates
        self.performance_by_intermediate(all_usage_data, strain_name, starter_made_date)

        # Overall performance charts
        self.pH_performance_analysis(all_usage_data, strain_name, starter_made_date)
        self.pH_distribution_analysis(all_usage_data, strain_name, starter_made_date)
        self.moisture_performance_analysis(all_usage_data, strain_name, starter_made_date)

        # Add process metrics by vat for the usage data
        self.process_metrics_by_vat(all_usage_data, strain_name, starter_made_date)

    def growth_metrics_summary(self, batch_row):
        """Display growth metrics summary for the starter batch in table format"""
        print("📊 STARTER GROWTH METRICS")
        print("-" * 30)

        metrics = {
            'Time to First Injection': f"{batch_row.get('TFI_minutes', 'N/A'):.1f} minutes" if pd.notna(batch_row.get('TFI_minutes')) else "N/A",
            'Average Interinjection Time': f"{batch_row.get('Avg. IIT_minutes', 'N/A'):.1f} minutes" if pd.notna(batch_row.get('Avg. IIT_minutes')) else "N/A",
            'Stdev Interinjection Time': f"{batch_row.get('Stdev IIT_minutes', 'N/A'):.1f} minutes" if pd.notna(batch_row.get('Stdev IIT_minutes')) else "N/A",
            'Min Interinjection Time': f"{batch_row.get('Min IIT_minutes', 'N/A'):.1f} minutes" if pd.notna(batch_row.get('Min IIT_minutes')) else "N/A",
            'Max Interinjection Time': f"{batch_row.get('Max IIT_minutes', 'N/A'):.1f} minutes" if pd.notna(batch_row.get('Max IIT_minutes')) else "N/A",
            'pH Change After Last Injection': f"{batch_row.get('pH CALI', 'N/A'):.2f}" if pd.notna(batch_row.get('pH CALI')) else "N/A"
        }

        # Create a formatted table
        print(f"{'Metric':<35} {'Value':<15}")
        print("-" * 50)
        for metric, value in metrics.items():
            print(f"{metric:<35} {value:<15}")
        print()

    def historical_comparison(self, batch_row, historical_data):
        """Compare current batch to historical strain performance in table format"""
        print("📈 HISTORICAL COMPARISON")
        print("-" * 25)

        metrics_to_compare = {
            'TFI_minutes': 'Time to First Injection',
            'Avg. IIT_minutes': 'Average Interinjection Time',
            'Stdev IIT_minutes': 'Stdev Interinjection Time',
            'Min IIT_minutes': 'Min Interinjection Time',
            'Max IIT_minutes': 'Max Interinjection Time',
            'pH CALI': 'pH Change After Last Injection'
        }

        # Create table headers
        print(f"{'Metric':<30} {'Current':<10} {'Hist Avg':<10} {'Hist Std':<10} {'Diff':<10} {'Z-Score':<10} {'Status':<6}")
        print("-" * 86)

        for metric_col, metric_name in metrics_to_compare.items():
            current_value = batch_row.get(metric_col)

            # For pH metrics, filter out zeros from historical data
            if 'pH' in metric_col:
                historical_values = historical_data[metric_col].dropna()
                historical_values = historical_values[historical_values != 0]
            else:
                historical_values = historical_data[metric_col].dropna()

            historical_avg = historical_values.mean()
            historical_std = historical_values.std()

            if pd.notna(current_value) and pd.notna(historical_avg) and current_value != 0:
                difference = current_value - historical_avg
                z_score = difference / historical_std if pd.notna(historical_std) and historical_std > 0 else 0

                # Process-specific flagging logic
                if 'IIT' in metric_col:  # Interinjection timing - only flag if too long
                    status = "🟢" if z_score < 1 else "🟡" if z_score < 2 else "🔴"
                elif metric_col == 'TFI_minutes':  # Time to First Injection
                    if z_score <= -5:  # Much too short
                        status = "🟡"
                    elif z_score < 1:  # Normal or slightly short
                        status = "🟢"
                    elif z_score < 2:  # Too long
                        status = "🟡"
                    else:  # Much too long
                        status = "🔴"
                elif metric_col == 'pH CALI':  # pH Change After Last Injection
                    if current_value >= -0.10:  # pH change >= -0.10 is bad
                        status = "🔴"
                    elif current_value > historical_avg:  # Current is higher (less negative) than average
                        if z_score > 2.0:  # Much higher (worse) than average
                            status = "🔴"
                        elif z_score > 1.0:  # Somewhat higher (worse) than average
                            status = "🟡"
                        else:  # Slightly higher than average but still acceptable
                            status = "🟢"
                    else:  # Current is lower (more negative) than average - potentially good
                        if z_score < -5.0:  # Much more negative than usual - flag as unusual
                            status = "🟡"
                        else:  # More negative than average - this is good!
                            status = "🟢"
                else:  # Default two-sided test
                    status = "🟢" if abs(z_score) < 1 else "🟡" if abs(z_score) < 2 else "🔴"

                # Format the table row
                short_metric = metric_name[:29] if len(metric_name) > 29 else metric_name
                print(f"{short_metric:<30} {current_value:<10.2f} {historical_avg:<10.2f} {historical_std:<10.2f} {difference:<+10.2f} {z_score:<10.2f} {status:<6}")

                if 'pH' in metric_col:
                    print(f"{'  (Based on ' + str(len(historical_values)) + ' non-zero measurements)':<86}")
            else:
                short_metric = metric_name[:29] if len(metric_name) > 29 else metric_name
                print(f"{short_metric:<30} {'N/A':<10} {'N/A':<10} {'N/A':<10} {'N/A':<10} {'N/A':<10} {'❌':<6}")
        print()

    def growth_metrics_trends(self, historical_data, strain_name, current_starter_date):
        """Create run charts for growth metrics over time with current batch highlighted"""
        print("📈 GROWTH METRICS TRENDS")
        print("-" * 26)

        growth_metrics = {
            'TFI_minutes': 'Time to First Injection (min)',
            'Avg. IIT_minutes': 'Average Interinjection Time (min)',
            'Stdev IIT_minutes': 'Stdev Interinjection Time (min)',
            'Min IIT_minutes': 'Min Interinjection Time (min)',
            'Max IIT_minutes': 'Max Interinjection Time (min)',
            'pH CALI': 'pH Change After Last Injection'
        }

        # Sort by starter made date for chronological order
        historical_data = historical_data.sort_values('Starter_Made_Date')

        available_metrics = [col for col in growth_metrics.keys() if col in historical_data.columns]

        if available_metrics:
            fig, axes = plt.subplots(len(available_metrics), 1, figsize=(12, 3*len(available_metrics)))
            if len(available_metrics) == 1:
                axes = [axes]

            # Set white background
            fig.patch.set_facecolor('white')

            for i, metric_col in enumerate(available_metrics):
                axes[i].set_facecolor('white')
                data = historical_data[['Starter_Made_Date', metric_col]].dropna()

                if len(data) > 1:
                    axes[i].plot(data['Starter_Made_Date'], data[metric_col], 'o-', linewidth=2, markersize=4, color='navy')

                    # Highlight the current batch date with a special marker
                    current_batch_data = data[data['Starter_Made_Date'] == current_starter_date]
                    if not current_batch_data.empty:
                        axes[i].scatter(current_batch_data['Starter_Made_Date'],
                                      current_batch_data[metric_col],
                                      s=150, marker='*', color='red', zorder=10,
                                      edgecolor='darkred', linewidth=2,
                                      label='Current Batch')

                    axes[i].set_ylabel(growth_metrics[metric_col])

                    # Light gray grid lines in background
                    axes[i].grid(True, alpha=0.4, color='gray', linestyle='-', linewidth=0.5)
                    axes[i].set_axisbelow(True)
                    axes[i].tick_params(axis='x', rotation=45)

                    # Format x-axis dates as MM-DD
                    import matplotlib.dates as mdates
                    axes[i].xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))

                    # Add overall average line
                    overall_avg = data[metric_col].mean()
                    axes[i].axhline(y=overall_avg, color='red', linestyle='--', alpha=0.8,
                                  label=f'Overall Avg: {overall_avg:.2f}')
                    axes[i].legend(loc='upper right')
                else:
                    axes[i].text(0.5, 0.5, 'Insufficient data', ha='center', va='center', transform=axes[i].transAxes)
                    axes[i].set_ylabel(growth_metrics[metric_col])
                    axes[i].set_facecolor('white')
                    axes[i].grid(True, alpha=0.4, color='gray', linestyle='-', linewidth=0.5)
                    axes[i].set_axisbelow(True)

            plt.suptitle(f'Growth Metrics Trends for {strain_name}', y=0.99)
            plt.tight_layout()
            # plt.savefig(f'growth_metrics_trends_{strain_name}.png', dpi=300, bbox_inches='tight')
            plt.show()
        else:
            print("  No growth metrics data available for plotting")
        print()

    def performance_by_intermediate(self, usage_data, strain_name, starter_date):
        """Analyze performance broken down by intermediate in table format"""
        if 'Intermediate' not in usage_data.columns:
            return

        print("🏭 PERFORMANCE BY INTERMEDIATE")
        print("-" * 35)

        intermediates = usage_data['Intermediate'].unique()

        if len(intermediates) == 1:
            print(f"  📦 Single intermediate: {intermediates[0]}")
            print(f"  📊 All {len(usage_data)} lots used same intermediate")
        else:
            # Create table format
            print(f"{'Intermediate':<40} {'Lots':<6} {'FG pH Avg':<10} {'FG pH Std':<10} {'FG Moist Avg':<12} {'FG Moist Std':<12}")
            print("-" * 102)

            for intermediate in sorted(intermediates):
                int_data = usage_data[usage_data['Intermediate'] == intermediate]

                # Clean up intermediate name
                if ' - ' in intermediate:
                    clean_name = intermediate.split(' - ', 1)[1]
                else:
                    clean_name = intermediate

                # Truncate if too long
                display_name = clean_name[:39] if len(clean_name) > 39 else clean_name

                lot_count = len(int_data)

                # FG pH statistics
                if 'FG pH' in int_data.columns:
                    ph_data = int_data['FG pH'][int_data['FG pH'] > 0]
                    if len(ph_data) > 0:
                        ph_avg = ph_data.mean()
                        ph_std = ph_data.std()
                        ph_avg_str = f"{ph_avg:.2f}"
                        ph_std_str = f"{ph_std:.2f}"
                    else:
                        ph_avg_str = "N/A"
                        ph_std_str = "N/A"
                else:
                    ph_avg_str = "N/A"
                    ph_std_str = "N/A"

                # FG Moisture statistics
                if 'FG Moisture' in int_data.columns:
                    moist_data = int_data['FG Moisture'][int_data['FG Moisture'] > 0]
                    if len(moist_data) > 0:
                        moist_avg = moist_data.mean()
                        moist_std = moist_data.std()
                        moist_avg_str = f"{moist_avg:.2f}%"
                        moist_std_str = f"{moist_std:.2f}%"
                    else:
                        moist_avg_str = "N/A"
                        moist_std_str = "N/A"
                else:
                    moist_avg_str = "N/A"
                    moist_std_str = "N/A"

                # Print table row
                print(f"{display_name:<40} {lot_count:<6} {ph_avg_str:<10} {ph_std_str:<10} {moist_avg_str:<12} {moist_std_str:<12}")
        print()

    def pH_performance_analysis(self, usage_data, strain_name, starter_date):
        """Create pH analysis across all usage of the starter batch"""
        print("🧪 pH PERFORMANCE ANALYSIS")
        print("-" * 30)

        ph_columns = ['pH to DMC', 'pH at First Turn', 'pH at Dice Mill', 'FG pH']

        # Filter out missing data (both NaN and zeros)
        ph_data = []
        labels = []
        for col in ph_columns:
            if col in usage_data.columns:
                data = usage_data[col].dropna()
                data = data[data > 0]  # Remove zeros
                if len(data) > 0:
                    ph_data.append(data)
                    labels.append(col)
                    print(f"  {col}: {len(data)} valid measurements (range: {data.min():.2f} - {data.max():.2f})")

        if ph_data:
            fig, ax = plt.subplots(figsize=(12, 8))

            # Set white background
            fig.patch.set_facecolor('white')
            ax.set_facecolor('white')

            # Create strip plot
            for i, data in enumerate(ph_data):
                # Add some random jitter for better visualization
                x_pos = [i + 1 + np.random.normal(0, 0.04) for _ in range(len(data))]
                ax.scatter(x_pos, data, alpha=0.6, s=50, color='navy', edgecolors='white', linewidth=0.5)

                # Add mean line
                mean_val = data.mean()
                ax.hlines(mean_val, i + 0.7, i + 1.3, colors='red', linestyles='-', linewidth=2, alpha=0.8)

                # Add median line (lighter)
                median_val = data.median()
                ax.hlines(median_val, i + 0.7, i + 1.3, colors='orange', linestyles='--', linewidth=1.5, alpha=0.7)

            ax.set_xticks(range(1, len(ph_data)+1))
            ax.set_xticklabels(labels, rotation=0)
            ax.set_title(f'pH Performance for {strain_name} Starter Made {starter_date.strftime("%m-%d")}')
            ax.set_ylabel('pH Value')

            # Light gray grid lines in background
            ax.grid(True, alpha=0.4, color='gray', linestyle='-', linewidth=0.5)
            ax.set_axisbelow(True)

            # Add legend
            from matplotlib.lines import Line2D
            legend_elements = [Line2D([0], [0], color='red', lw=2, label='Mean'),
                             Line2D([0], [0], color='orange', lw=1.5, linestyle='--', label='Median')]
            ax.legend(handles=legend_elements, loc='upper right')

            plt.tight_layout()
            # plt.savefig(f'pH_performance_{strain_name}_{starter_date.strftime("%m%d")}.png', dpi=300, bbox_inches='tight')
            plt.show()
        else:
            print("  No pH data available for plotting")
        print()

    def pH_distribution_analysis(self, usage_data, strain_name, starter_date):
        """Create finished goods pH distribution analysis with specification limits"""
        print("🧪 FINISHED GOODS pH ANALYSIS")
        print("-" * 32)

        if 'FG pH' not in usage_data.columns:
            print("  ❌ 'FG pH' column not found in data")
            return

        # Get FG pH data for distribution
        fg_ph_data = usage_data['FG pH'].dropna()
        fg_ph_data = fg_ph_data[fg_ph_data > 0]  # Remove zeros

        if len(fg_ph_data) == 0:
            print("  No valid FG pH data available")
            return

        print(f"  📊 {len(fg_ph_data)} valid FG pH measurements")
        print(f"  📈 Range: {fg_ph_data.min():.2f} - {fg_ph_data.max():.2f}")
        print(f"  📈 Average: {fg_ph_data.mean():.2f} ± {fg_ph_data.std():.2f}")

        # Create side-by-side plots (distribution + by intermediate if available)
        if 'Intermediate' in usage_data.columns and len(usage_data['Intermediate'].unique()) > 1:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), gridspec_kw={'width_ratios': [1, 2]})
        else:
            fig, ax1 = plt.subplots(1, 1, figsize=(6, 6))
            ax2 = None

        # Set white background
        fig.patch.set_facecolor('white')
        ax1.set_facecolor('white')

        # Left plot: Strip plot distribution
        x_pos = [1 + np.random.normal(0, 0.02) for _ in range(len(fg_ph_data))]
        ax1.scatter(x_pos, fg_ph_data, alpha=0.6, s=50, color='navy', edgecolors='white', linewidth=0.5)

        # Add mean and median lines
        mean_val = fg_ph_data.mean()
        median_val = fg_ph_data.median()
        ax1.hlines(mean_val, 0.8, 1.2, colors='red', linestyles='-', linewidth=2, alpha=0.8)
        ax1.hlines(median_val, 0.8, 1.2, colors='orange', linestyles='--', linewidth=1.5, alpha=0.7)

        # Add overall specification limits if available
        if 'pH_Max_Spec' in usage_data.columns and 'pH_Min_Spec' in usage_data.columns:
            ph_specs = usage_data[['pH_Max_Spec', 'pH_Min_Spec']].dropna()
            if not ph_specs.empty:
                overall_max = ph_specs['pH_Max_Spec'].max()
                overall_min = ph_specs['pH_Min_Spec'].min()
                ax1.hlines(overall_max, 0.7, 1.3, colors='red', linestyles=':', linewidth=2, alpha=0.9)
                ax1.hlines(overall_min, 0.7, 1.3, colors='red', linestyles=':', linewidth=2, alpha=0.9)
                ax1.text(1.35, overall_max, f'Max: {overall_max:.2f}', va='center', fontsize=8, color='red')
                ax1.text(1.35, overall_min, f'Min: {overall_min:.2f}', va='center', fontsize=8, color='red')

        ax1.set_xticks([1])
        ax1.set_xticklabels(['FG pH'])
        ax1.set_title('FG pH Distribution')
        ax1.set_ylabel('pH Value')
        ax1.set_xlim(0.6, 1.4)

        # Grid and legend
        ax1.grid(True, alpha=0.4, color='gray', linestyle='-', linewidth=0.5)
        ax1.set_axisbelow(True)

        from matplotlib.lines import Line2D
        legend_elements = [Line2D([0], [0], color='red', lw=2, label='Mean'),
                         Line2D([0], [0], color='orange', lw=1.5, linestyle='--', label='Median')]
        if 'pH_Max_Spec' in usage_data.columns:
            legend_elements.append(Line2D([0], [0], color='red', lw=2, linestyle=':', label='Spec Limits'))
        ax1.legend(handles=legend_elements, loc='upper right')

        # Right plot: By intermediate (if multiple intermediates exist)
        if ax2 is not None:
            ax2.set_facecolor('white')
            intermediates = sorted(usage_data['Intermediate'].unique())

            int_data = []
            int_labels = []
            for intermediate in intermediates:
                int_fg_ph = usage_data[usage_data['Intermediate'] == intermediate]['FG pH']
                int_fg_ph = int_fg_ph.dropna()
                int_fg_ph = int_fg_ph[int_fg_ph > 0]
                if len(int_fg_ph) > 0:
                    int_data.append(int_fg_ph)
                    # Clean up label - remove IM Item No, keep only description
                    if ' - ' in intermediate:
                        desc_only = intermediate.split(' - ', 1)[1]
                    else:
                        desc_only = intermediate
                    int_labels.append(desc_only)

            if int_data:
                for i, data in enumerate(int_data):
                    x_pos = [i + 1 + np.random.normal(0, 0.02) for _ in range(len(data))]
                    ax2.scatter(x_pos, data, alpha=0.6, s=50, color='navy', edgecolors='white', linewidth=0.5)

                    # Add mean line for each intermediate
                    mean_val = data.mean()
                    ax2.hlines(mean_val, i + 0.8, i + 1.2, colors='red', linestyles='-', linewidth=2, alpha=0.8)

                    # Add mean value annotation next to the data cluster
                    ax2.text(i + 1.3, mean_val, f'{mean_val:.2f}', va='center', ha='left',
                            fontsize=10, color='black', weight='bold')

                    # Add specification limits for this intermediate
                    intermediate_name = intermediates[i]
                    int_specs = usage_data[usage_data['Intermediate'] == intermediate_name][['pH_Max_Spec', 'pH_Min_Spec']].dropna()
                    if not int_specs.empty and len(int_specs) > 0:
                        spec_max = int_specs['pH_Max_Spec'].iloc[0]
                        spec_min = int_specs['pH_Min_Spec'].iloc[0]
                        if pd.notna(spec_max) and pd.notna(spec_min):
                            # Add specification limit lines
                            ax2.hlines(spec_max, i + 0.7, i + 1.3, colors='red', linestyles=':', linewidth=2, alpha=0.9)
                            ax2.hlines(spec_min, i + 0.7, i + 1.3, colors='red', linestyles=':', linewidth=2, alpha=0.9)

                            # Add spec limit labels
                            ax2.text(i + 1.35, spec_max, f'{spec_max:.2f}', va='center', ha='left',
                                    fontsize=8, color='red', style='italic')
                            ax2.text(i + 1.35, spec_min, f'{spec_min:.2f}', va='center', ha='left',
                                    fontsize=8, color='red', style='italic')

                ax2.set_xticks(range(1, len(int_data)+1))
                ax2.set_xticklabels(int_labels, rotation=0, ha='center')  # Horizontal labels
                ax2.set_title('FG pH by Intermediate')
                ax2.set_ylabel('pH Value')

                ax2.grid(True, alpha=0.4, color='gray', linestyle='-', linewidth=0.5)
                ax2.set_axisbelow(True)

        plt.tight_layout()
        # plt.savefig(f'FG_pH_analysis_{strain_name}_{starter_date.strftime("%m%d")}.png', dpi=300, bbox_inches='tight')
        plt.show()
        print()

    def moisture_performance_analysis(self, usage_data, strain_name, starter_date):
        """Create moisture analysis with specification limits across all usage of the starter batch"""
        print("💧 MOISTURE PERFORMANCE ANALYSIS")
        print("-" * 35)

        if 'FG Moisture' not in usage_data.columns:
            print("  ❌ 'FG Moisture' column not found in data")
            return

        raw_moisture = usage_data['FG Moisture']
        moisture_data = raw_moisture.dropna()
        moisture_data = moisture_data[moisture_data > 0]  # Remove zeros

        if len(moisture_data) == 0:
            print("  No valid moisture data")
            return

        print(f"  📊 {len(moisture_data)} valid moisture measurements")
        print(f"  📈 Range: {moisture_data.min():.2f} - {moisture_data.max():.2f}%")
        print(f"  📈 Average: {moisture_data.mean():.2f}% ± {moisture_data.std():.2f}%")

        # Create side-by-side plots (distribution + by intermediate if available)
        if 'Intermediate' in usage_data.columns and len(usage_data['Intermediate'].unique()) > 1:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), gridspec_kw={'width_ratios': [1, 2]})
        else:
            fig, ax1 = plt.subplots(1, 1, figsize=(6, 6))
            ax2 = None

        # Set white background
        fig.patch.set_facecolor('white')
        ax1.set_facecolor('white')

        # Left plot: Strip plot distribution
        x_pos = [1 + np.random.normal(0, 0.02) for _ in range(len(moisture_data))]
        ax1.scatter(x_pos, moisture_data, alpha=0.6, s=50, color='darkgreen', edgecolors='white', linewidth=0.5)

        # Add mean and median lines
        mean_val = moisture_data.mean()
        median_val = moisture_data.median()
        ax1.hlines(mean_val, 0.8, 1.2, colors='red', linestyles='-', linewidth=2, alpha=0.8)
        ax1.hlines(median_val, 0.8, 1.2, colors='orange', linestyles='--', linewidth=1.5, alpha=0.7)

        # Add overall specification limits if available
        if 'Moisture_Max_Spec' in usage_data.columns and 'Moisture_Min_Spec' in usage_data.columns:
            moist_specs = usage_data[['Moisture_Max_Spec', 'Moisture_Min_Spec']].dropna()
            if not moist_specs.empty:
                overall_max = moist_specs['Moisture_Max_Spec'].max()
                overall_min = moist_specs['Moisture_Min_Spec'].min()
                ax1.hlines(overall_max, 0.7, 1.3, colors='red', linestyles=':', linewidth=2, alpha=0.9)
                ax1.hlines(overall_min, 0.7, 1.3, colors='red', linestyles=':', linewidth=2, alpha=0.9)
                ax1.text(1.35, overall_max, f'Max: {overall_max:.1f}%', va='center', fontsize=8, color='red')
                ax1.text(1.35, overall_min, f'Min: {overall_min:.1f}%', va='center', fontsize=8, color='red')

        ax1.set_xticks([1])
        ax1.set_xticklabels(['Moisture'])
        ax1.set_title('Moisture Distribution')
        ax1.set_ylabel('Moisture (%)')
        ax1.set_xlim(0.6, 1.4)

        # Grid and legend
        ax1.grid(True, alpha=0.4, color='gray', linestyle='-', linewidth=0.5)
        ax1.set_axisbelow(True)

        from matplotlib.lines import Line2D
        legend_elements = [Line2D([0], [0], color='red', lw=2, label='Mean'),
                         Line2D([0], [0], color='orange', lw=1.5, linestyle='--', label='Median')]
        if 'Moisture_Max_Spec' in usage_data.columns:
            legend_elements.append(Line2D([0], [0], color='red', lw=2, linestyle=':', label='Spec Limits'))
        ax1.legend(handles=legend_elements, loc='upper right')

        # Right plot: By intermediate (if multiple intermediates exist)
        if ax2 is not None:
            ax2.set_facecolor('white')
            intermediates = sorted(usage_data['Intermediate'].unique())

            int_data = []
            int_labels = []
            for intermediate in intermediates:
                int_moisture = usage_data[usage_data['Intermediate'] == intermediate]['FG Moisture']
                int_moisture = int_moisture.dropna()
                int_moisture = int_moisture[int_moisture > 0]
                if len(int_moisture) > 0:
                    int_data.append(int_moisture)
                    # Clean up label - remove IM Item No and (n=x), keep only description
                    if ' - ' in intermediate:
                        desc_only = intermediate.split(' - ', 1)[1]  # Take everything after first " - "
                    else:
                        desc_only = intermediate
                    int_labels.append(desc_only)

            if int_data:
                for i, data in enumerate(int_data):
                    x_pos = [i + 1 + np.random.normal(0, 0.02) for _ in range(len(data))]
                    ax2.scatter(x_pos, data, alpha=0.6, s=50, color='darkgreen', edgecolors='white', linewidth=0.5)

                    # Add mean line for each intermediate
                    mean_val = data.mean()
                    ax2.hlines(mean_val, i + 0.8, i + 1.2, colors='red', linestyles='-', linewidth=2, alpha=0.8)

                    # Add mean value annotation next to the data cluster
                    ax2.text(i + 1.3, mean_val, f'{mean_val:.2f}%', va='center', ha='left',
                            fontsize=10, color='black', weight='bold')

                    # Add specification limits for this intermediate
                    intermediate_name = intermediates[i]
                    int_specs = usage_data[usage_data['Intermediate'] == intermediate_name][['Moisture_Max_Spec', 'Moisture_Min_Spec']].dropna()
                    if not int_specs.empty and len(int_specs) > 0:
                        spec_max = int_specs['Moisture_Max_Spec'].iloc[0]
                        spec_min = int_specs['Moisture_Min_Spec'].iloc[0]
                        if pd.notna(spec_max) and pd.notna(spec_min):
                            # Add specification limit lines
                            ax2.hlines(spec_max, i + 0.7, i + 1.3, colors='red', linestyles=':', linewidth=2, alpha=0.9)
                            ax2.hlines(spec_min, i + 0.7, i + 1.3, colors='red', linestyles=':', linewidth=2, alpha=0.9)

                            # Add spec limit labels
                            ax2.text(i + 1.35, spec_max, f'{spec_max:.1f}%', va='center', ha='left',
                                    fontsize=8, color='red', style='italic')
                            ax2.text(i + 1.35, spec_min, f'{spec_min:.1f}%', va='center', ha='left',
                                    fontsize=8, color='red', style='italic')

                ax2.set_xticks(range(1, len(int_data)+1))
                ax2.set_xticklabels(int_labels, rotation=0, ha='center')  # Horizontal labels
                ax2.set_title('Moisture by Intermediate')
                ax2.set_ylabel('Moisture (%)')

                ax2.grid(True, alpha=0.4, color='gray', linestyle='-', linewidth=0.5)
                ax2.set_axisbelow(True)

        plt.tight_layout()
        # plt.savefig(f'moisture_analysis_{strain_name}_{starter_date.strftime("%m%d")}.png', dpi=300, bbox_inches='tight')
        plt.show()
        print()

    def process_metrics_by_vat(self, usage_data, strain_name, starter_date):
        """Create charts showing process metrics performance by vat with enhanced intermediate handling"""
        print("🏭 PROCESS METRICS BY VAT")
        print("-" * 28)

        # Extract vat number from Lot Number (last 2 digits)
        def extract_vat_from_lot(lot_number):
            try:
                lot_str = str(int(lot_number))  # Convert to string, removing any decimals
                if len(lot_str) >= 2:
                    return int(lot_str[-2:])  # Last 2 digits
                else:
                    return None
            except:
                return None

        vat_data = usage_data.copy()
        vat_data['Vat'] = vat_data['Lot Number'].apply(extract_vat_from_lot)
        vat_data = vat_data.dropna(subset=['Vat'])

        if vat_data.empty:
            print("  Could not extract vat numbers from lot numbers")
            return

        # Smart ordering for cross-day production with date tracking
        # Sort vats intelligently to handle cross-day sequences like [46,47,48,49,50,51,1,2,3...]
        sorted_vats = sorted(vat_data['Vat'].unique())

        # Check if we have a cross-day pattern (high vats followed by low vats)
        cross_day_detected = False
        vat_labels = []  # Will store vat labels with dates if needed

        if len(sorted_vats) > 1:
            # Look for gaps that suggest day transition
            gaps = []
            for i in range(1, len(sorted_vats)):
                gap = sorted_vats[i] - sorted_vats[i-1]
                gaps.append((gap, i, sorted_vats[i-1], sorted_vats[i]))

            # Find largest negative gap (suggests day rollover)
            largest_negative_gap = None
            for gap, idx, prev_vat, curr_vat in gaps:
                if gap < -20:  # Significant negative jump (e.g., 51 -> 1)
                    if largest_negative_gap is None or gap < largest_negative_gap[0]:
                        largest_negative_gap = (gap, idx, prev_vat, curr_vat)

            # If we found a day transition, reorder the vats chronologically
            if largest_negative_gap:
                split_idx = largest_negative_gap[1]
                # Put the later vats (after transition) first, then early vats
                reordered_vats = sorted_vats[split_idx:] + sorted_vats[:split_idx]

                # Create a custom sort order
                vat_order = {vat: idx for idx, vat in enumerate(reordered_vats)}
                vat_data['Sort_Order'] = vat_data['Vat'].map(vat_order)
                vat_data = vat_data.sort_values('Sort_Order')
                cross_day_detected = True

                # Create vat labels with dates for cross-day production
                for i, vat in enumerate(reordered_vats):
                    vat_row = vat_data[vat_data['Vat'] == vat].iloc[0]
                    if 'DOM' in vat_row and pd.notna(vat_row['DOM']):
                        date_str = pd.to_datetime(vat_row['DOM']).strftime('%m/%d')
                        vat_labels.append(f"{vat}\n{date_str}")
                    else:
                        vat_labels.append(str(vat))
            else:
                # Normal sorting by vat number
                vat_data = vat_data.sort_values('Vat')
                vat_labels = [str(vat) for vat in sorted_vats]
        else:
            vat_data = vat_data.sort_values('Vat')
            vat_labels = [str(vat) for vat in sorted_vats]

        process_metrics = {
            'Starter QTY': 'Starter Quantity',
            'pH to DMC': 'pH to DMC',
            'pH at First Turn': 'pH at First Turn',
            'pH at Dice Mill': 'pH at Dice Mill',
            'FG pH': 'Finished Goods pH'
        }

        available_metrics = [col for col in process_metrics.keys() if col in vat_data.columns]

        if available_metrics:
            fig, axes = plt.subplots(len(available_metrics), 1, figsize=(12, 3*len(available_metrics)))
            if len(available_metrics) == 1:
                axes = [axes]

            # Set white background
            fig.patch.set_facecolor('white')

            # Determine intermediate groupings based on Starter QTY (most complete field)
            intermediate_ranges = {}
            if 'Starter QTY' in vat_data.columns:
                # Use Starter QTY to identify intermediate groups since it has fewer missing values
                starter_qty_data = vat_data[['Vat', 'Starter QTY', 'Intermediate']].dropna(subset=['Starter QTY'])

                if not starter_qty_data.empty:
                    for _, row in starter_qty_data.iterrows():
                        intermediate = row['Intermediate']
                        vat = row['Vat']

                        # Clean up intermediate name
                        if ' - ' in intermediate:
                            clean_name = intermediate.split(' - ', 1)[1]
                        else:
                            clean_name = intermediate

                        if clean_name not in intermediate_ranges:
                            intermediate_ranges[clean_name] = []
                        intermediate_ranges[clean_name].append(vat)

                    # Sort vat ranges within each intermediate
                    for intermediate in intermediate_ranges:
                        intermediate_ranges[intermediate] = sorted(list(set(intermediate_ranges[intermediate])))

            for i, metric_col in enumerate(available_metrics):
                axes[i].set_facecolor('white')

                # For pH columns, filter out zeros
                plot_data = vat_data[['Vat', metric_col, 'Intermediate']].copy()
                if 'pH' in metric_col:
                    plot_data = plot_data[plot_data[metric_col] > 0]

                plot_data = plot_data.dropna()

                if len(plot_data) > 0:
                    axes[i].plot(plot_data['Vat'], plot_data[metric_col], 'o-', linewidth=2, markersize=6, color='navy')
                    axes[i].set_ylabel(process_metrics[metric_col])

                    # Light gray grid lines in background
                    axes[i].grid(True, alpha=0.4, color='gray', linestyle='-', linewidth=0.5)
                    axes[i].set_axisbelow(True)

                    # Set x-axis to show all vat numbers with proper labels
                    if len(plot_data) > 1:
                        vat_list = plot_data['Vat'].tolist()
                        axes[i].set_xticks(vat_list)

                        # Use date labels if cross-day detected, otherwise just vat numbers
                        if cross_day_detected:
                            display_labels = []
                            for vat in vat_list:
                                vat_row = vat_data[vat_data['Vat'] == vat].iloc[0]
                                if 'DOM' in vat_row and pd.notna(vat_row['DOM']):
                                    date_str = pd.to_datetime(vat_row['DOM']).strftime('%m/%d')
                                    display_labels.append(f"{vat}\n{date_str}")
                                else:
                                    display_labels.append(str(vat))
                            axes[i].set_xticklabels(display_labels, fontsize=8)
                        else:
                            axes[i].set_xticklabels([str(vat) for vat in vat_list])

                    # Add intermediate-specific reference lines (ALL RED)
                    if intermediate_ranges and 'pH' in metric_col:
                        for intermediate, vats in intermediate_ranges.items():
                            # Get data for this intermediate
                            int_data = plot_data[plot_data['Vat'].isin(vats)][metric_col]

                            if len(int_data) > 1:
                                int_avg = int_data.mean()

                                # Draw reference line only for vats in this intermediate (ALL RED)
                                min_vat = min([v for v in plot_data['Vat'] if v in vats])
                                max_vat = max([v for v in plot_data['Vat'] if v in vats])

                                axes[i].hlines(int_avg, min_vat, max_vat,
                                             colors='red', linestyles='--', alpha=0.8, linewidth=2)

                                # Add small data label next to reference line (RED)
                                label_x = min_vat + (max_vat - min_vat) * 0.15
                                axes[i].text(label_x, int_avg, f'{int_avg:.2f}',
                                           fontsize=8, color='red', weight='bold',
                                           ha='left', va='bottom',
                                           bbox=dict(boxstyle='round,pad=0.2', facecolor='white',
                                                   edgecolor='red', alpha=0.8))
                    elif not 'pH' in metric_col:
                        # For non-pH metrics (like Starter QTY), add overall average (RED)
                        if len(plot_data) > 1:
                            avg_value = plot_data[metric_col].mean()
                            axes[i].axhline(y=avg_value, color='red', linestyle='--', alpha=0.8)
                            axes[i].text(plot_data['Vat'].iloc[0], avg_value, f'Avg: {avg_value:.2f}',
                                       fontsize=8, color='red', weight='bold', ha='left', va='bottom',
                                       bbox=dict(boxstyle='round,pad=0.2', facecolor='white',
                                               edgecolor='red', alpha=0.8))

                    # Add intermediate background shading
                    if intermediate_ranges:
                        bg_colors = ['blue', 'green', 'orange', 'purple', 'brown']
                        color_idx = 0

                        for intermediate, vats in intermediate_ranges.items():
                            if len(vats) > 0:
                                min_vat = min(vats)
                                max_vat = max(vats)
                                bg_color = bg_colors[color_idx % len(bg_colors)]

                                # Add colored background shading for this intermediate
                                axes[i].axvspan(min_vat - 0.4, max_vat + 0.4, alpha=0.1,
                                              color=bg_color, zorder=0)
                                color_idx += 1
                else:
                    axes[i].text(0.5, 0.5, 'No valid data', ha='center', va='center',
                               transform=axes[i].transAxes)
                    axes[i].set_ylabel(process_metrics[metric_col])
                    axes[i].set_facecolor('white')
                    axes[i].grid(True, alpha=0.4, color='gray', linestyle='-', linewidth=0.5)
                    axes[i].set_axisbelow(True)

                # Set xlabel only on bottom chart
                if i == len(available_metrics) - 1:
                    if cross_day_detected:
                        axes[i].set_xlabel('Vat Number\n(with Production Date)')
                    else:
                        axes[i].set_xlabel('Vat Number')

            # Add intermediate labels ABOVE the entire chart area (outside plot area)
            if intermediate_ranges:
                bg_colors = ['blue', 'green', 'orange', 'purple', 'brown']
                color_idx = 0

                # Use the top axis to add lines above the chart area
                top_ax = axes[0]  # Use the top chart for positioning

                # Get current y-limits to position labels above
                y_min, y_max = top_ax.get_ylim()
                y_range = y_max - y_min
                label_line_y = y_max + y_range * 0.15  # Position for the line
                label_text_y = y_max + y_range * 0.20  # Position for the text

                for intermediate, vats in intermediate_ranges.items():
                    if len(vats) > 0:
                        min_vat = min(vats)
                        max_vat = max(vats)
                        center_vat = (min_vat + max_vat) / 2
                        bg_color = bg_colors[color_idx % len(bg_colors)]

                        # Add colored line that spans exactly the vat range
                        top_ax.plot([min_vat, max_vat], [label_line_y, label_line_y],
                                  color=bg_color, linewidth=4, solid_capstyle='butt')

                        # Add intermediate name above the line
                        top_ax.text(center_vat, label_text_y, intermediate,
                                  ha='center', va='bottom', fontsize=10,
                                  color=bg_color, weight='bold')

                        color_idx += 1

                # Extend the y-axis of the top chart to accommodate labels
                top_ax.set_ylim(y_min, y_max + y_range * 0.25)

            plt.suptitle(f'Process Metrics by Vat for {strain_name} Starter Made {starter_date.strftime("%m-%d")}', y=1.005)
            plt.tight_layout(rect=[0, 0.03, 1, 0.99])
            # plt.savefig(f'process_metrics_by_vat_{strain_name}_{starter_date.strftime("%m%d")}.png', dpi=300, bbox_inches='tight')
            plt.show()

            print(f"  📊 Analyzed {len(vat_data)} lots across {vat_data['Vat'].nunique()} vats")
            if cross_day_detected:
                print(f"  🔄 Detected cross-day production pattern - vats reordered chronologically with dates")
        else:
            print("  No process metrics data available for plotting")
        print()

# Usage function
def create_batch_analyzer(file_path="/content/drive/MyDrive/FFUSA/RC Starter Tank Monitoring/26-02-starter-analysis v2.xlsx",
                         recipe_file_path="/content/drive/MyDrive/FFUSA/RC Starter Tank Monitoring/recipe_export_rc_im.xlsx",
                         debug_batch_ids=False):
    """
    Create and display the starter batch analyzer with optional recipe specifications
    """
    try:
        print("🚀 Loading Starter Batch Analyzer...")
        analyzer = StarterBatchAnalyzer(file_path, recipe_file_path)

        print(f"✅ Data loaded and processed successfully!")
        print(f"   📊 Total records: {len(analyzer.df)}")

        # Check if we have valid dates and automatically debug if not
        if 'Starter_Made_Date' in analyzer.df.columns:
            valid_dates = analyzer.df['Starter_Made_Date'].dropna()
            datetime_dates = [d for d in valid_dates if pd.notna(d) and isinstance(d, (pd.Timestamp, datetime))]

            if len(datetime_dates) > 0:
                print(f"   📅 Starter date range: {min(datetime_dates).strftime('%Y-%m-%d')} to {max(datetime_dates).strftime('%Y-%m-%d')}")
            else:
                print("   ⚠️ No valid starter dates found - running automatic debug...")
                debug_batch_ids = True

        # Debug batch IDs if requested or if there are issues
        if debug_batch_ids:
            analyzer.debug_batch_ids()

        print(f"   🧪 Unique strains: {analyzer.df['Starter Strain'].nunique()}")
        if 'Intermediate' in analyzer.df.columns:
            print(f"   🏭 Unique intermediates: {analyzer.df['Intermediate'].nunique()}")
        print()

        # Display the interactive interface
        analyzer.display_interface()

        return analyzer

    except FileNotFoundError:
        print("❌ File not found. Please check your file path.")
        print("Make sure you've mounted Google Drive and the file path is correct.")
        return None
    except Exception as e:
        print(f"❌ Error: {e}")
        print("\n🔍 Running diagnostic debug...")
        try:
            # Try to load the file and run debug
            analyzer = StarterBatchAnalyzer(file_path, recipe_file_path)
            analyzer.debug_batch_ids()
        except Exception as debug_e:
            print(f"Debug also failed: {debug_e}")
            import traceback
            print("Full error details:")
            traceback.print_exc()
        return None

# Example usage
if __name__ == "__main__":
    # Create the batch analyzer
    analyzer = create_batch_analyzer()

🚀 Loading Starter Batch Analyzer...
✅ Recipe specifications loaded: 23 intermediates
🔍 Available columns in data file:
   ['Lot Number', 'DOM', 'Vat No', 'IM Item No', 'IM Item Desc', 'Starter No', 'Starter Strain', 'Starter Tank', 'Starter QTY', 'Starter Batch ID', 'pH to DMC', 'pH at First Turn', 'pH at Dice Mill', 'TFI', 'Avg IIT', 'Stdev IIT', 'Min IIT', 'Max IIT', 'pH CALI', 'FG Moisture', 'FG pH']
✅ Intermediate column created using 'IM Item No' and 'IM Item Desc'
   Sample intermediates: ['10000688 - MOZZ LMPS 40 INTERMEDIATE', '10000691 - MOZZ WM 46.5 INTERMEDIATE', '10007338 - MOZZ WM 47 ASADERO INTERMEDIATE', '10000687 - MOZZ LMPS 38 INTERMEDIATE', '10008159 - MOZZ LMPS 39.25 PRECISION INTERMEDIATE']
🔍 Recipe specs columns: ['im_item_no', 'im_item_desc', 'im_moi_max', 'im_moi_min', 'im_ph_max', 'im_ph_min']
📊 Original data records: 1335
📊 Recipe spec records: 23
   ⚠️  Removed 11 duplicate recipe specs
📊 Main data unique item numbers: 9
📊 Recipe spec unique item numbers: 12
📊